In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 59.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=b326cb64245fc0d93ae01dfe7a07a253bf4b5d9ae70131b0b5116f356bc1c281
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


# DBS Ratna: Script2

p40


- data inv_CUSTMAST;
- input BP_S_NUMBER $ CIN_SFX $;
- datalines;
- ab12 mn-34
- cd45 jk-45
- kl56 gh-67
- ;
- run;

- data DATAMART;
- input PARTY_ID newcinsfx $;
- datalines;
- 12 mn34
- 45 gh67
- 13 kj89
- 44 ik90
- ;
- run;


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *

#table:inv_CUSTMAST
data= [["ab12", "mn-34"],["cd45", "jk-45"],["kl56", "gh-67"], [' ', 'kl-87']]
columns=["BP_S_NUMBER", "CIN_SFX"]
spark.conf.set("CUSTMAST",10)

spark = SparkSession.builder.appName('sas2py').getOrCreate()
inv_custmast=spark.createDataFrame(data=data,schema=columns)
inv_custmast.show()
inv_custmast.createOrReplaceTempView("inv_10")


#table:DATAMART
data= [[12, "mn34"],[45, "gh67"],[13, "kj89"], [44, "ik90"]]
columns=["PARTY_ID", "newcinsfx"]
spark.conf.set("DATAMART",'DATAMART')

spark = SparkSession.builder.appName('sas2py').getOrCreate()
DATAMART=spark.createDataFrame(data=data,schema=columns)
DATAMART.show()
DATAMART.createOrReplaceTempView("DATAMART")

+-----------+-------+
|BP_S_NUMBER|CIN_SFX|
+-----------+-------+
|       ab12|  mn-34|
|       cd45|  jk-45|
|       kl56|  gh-67|
|           |  kl-87|
+-----------+-------+

+--------+---------+
|PARTY_ID|newcinsfx|
+--------+---------+
|      12|     mn34|
|      45|     gh67|
|      13|     kj89|
|      44|     ik90|
+--------+---------+



In [ ]:
avaloq_curr = spark.sql("""
select a.BP_S_NUMBER, a.CIN_SFX, b.PARTY_ID from inv_${CUSTMAST} a inner join ${DATAMART}  b on b.newcinsfx = replace(a.CIN_SFX,'-', '')
where a.CIN_SFX != '' and a.BP_S_NUMBER != ''
""")

avaloq_curr.show()


+-----------+-------+--------+
|BP_S_NUMBER|CIN_SFX|PARTY_ID|
+-----------+-------+--------+
|       kl56|  gh-67|      45|
|       ab12|  mn-34|      12|
+-----------+-------+--------+



### p40 General Way

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *
#table:inv_CUSTMAST
data= [["ab12", "mn-34"],["cd45", "jk-45"],["kl56", "gh-67"], [' ', 'kl-87']]
columns=["BP_S_NUMBER", "CIN_SFX"]

spark = SparkSession.builder.appName('sas2py').getOrCreate()
inv_custmast=spark.createDataFrame(data=data,schema=columns)
inv_custmast.show()
inv_custmast.createOrReplaceTempView("inv_custmast")

#table:DATAMART
data= [[12, "mn34"],[45, "gh67"],[13, "kj89"], [44, "ik90"]]
columns=["PARTY_ID", "newcinsfx"]

spark = SparkSession.builder.appName('sas2py').getOrCreate()
DATAMART=spark.createDataFrame(data=data,schema=columns)
DATAMART.show()
DATAMART.createOrReplaceTempView("DATAMART")

+-----------+-------+
|BP_S_NUMBER|CIN_SFX|
+-----------+-------+
|       ab12|  mn-34|
|       cd45|  jk-45|
|       kl56|  gh-67|
|           |  kl-87|
+-----------+-------+

+--------+---------+
|PARTY_ID|newcinsfx|
+--------+---------+
|      12|     mn34|
|      45|     gh67|
|      13|     kj89|
|      44|     ik90|
+--------+---------+



In [ ]:
from pyspark.sql.functions import regexp_replace

"""
inv_CUSTMAST = inv_CUSTMAST.select(["CIN_SFX", "BP_S_NUMBER"])
inv_CUSTMAST = inv_CUSTMAST.filter((inv_CUSTMAST.CIN_SFX != ' ') & (inv_CUSTMAST.BP_S_NUMBER != ' '))
inv_CUSTMAST.createOrReplaceTempView("inv_CUSTMAST")
DATAMART = DATAMART.select(["newcinsfx", "PARTY_ID"])
DATAMART.createOrReplaceTempView("DATAMART")
"""
spark.sql("""

select a.BP_S_NUMBER, a.CIN_SFX, b.PARTY_ID from inv_CUSTMAST a inner join DATAMART b on b.newcinsfx = replace(a.CIN_SFX,'-', '') 
where a.CIN_SFX != '' and a.BP_S_NUMBER != ''

""").show()

+-----------+-------+--------+
|BP_S_NUMBER|CIN_SFX|PARTY_ID|
+-----------+-------+--------+
|       kl56|  gh-67|      45|
|       ab12|  mn-34|      12|
+-----------+-------+--------+



### p41

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *

data=[[1,5,7],[2,8,7]]
columns=['PARTY_ID','BP_S_NUMBER','S_Account']
txn2_curr= spark.createDataFrame(data=data,schema=columns)
txn2_curr.show()
txn2_curr.createOrReplaceTempView('txn2_curr')
data=[[1,4,3],[1,2,5]]
columns=['PARTY_ID','BP_S_NUMBER','S_Account']
avaloq_curr= spark.createDataFrame(data=data,schema=columns)
avaloq_curr.show()
avaloq_curr.createOrReplaceTempView('avaloq_curr')

+--------+-----------+---------+
|PARTY_ID|BP_S_NUMBER|S_Account|
+--------+-----------+---------+
|       1|          5|        7|
|       2|          8|        7|
+--------+-----------+---------+

+--------+-----------+---------+
|PARTY_ID|BP_S_NUMBER|S_Account|
+--------+-----------+---------+
|       1|          4|        3|
|       1|          2|        5|
+--------+-----------+---------+



In [ ]:
txn3_curr=spark.sql("""
select coalesce(b.PARTY_ID,a.PARTY_ID) as PARTY_ID , a.* from txn2_curr a left join avaloq_curr b on b.BP_S_NUMBER = a.S_Account
""")

txn3_curr.show()

+--------+--------+-----------+---------+
|PARTY_ID|PARTY_ID|BP_S_NUMBER|S_Account|
+--------+--------+-----------+---------+
|       1|       1|          5|        7|
|       2|       2|          8|        7|
+--------+--------+-----------+---------+



### p42

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *

columns = ["PARTY_ID", "pdtGroup"]
data = [(1,23), (1,23),(8,24),(8,24)]
txn3_curr= spark.createDataFrame(data=data,schema=columns)
txn3_curr.show()
txn3_curr.createOrReplaceTempView('txn3_curr')

+--------+--------+
|PARTY_ID|pdtGroup|
+--------+--------+
|       1|      23|
|       1|      23|
|       8|      24|
|       8|      24|
+--------+--------+



In [ ]:
txn5_curr=spark.sql("""select distinct PARTY_ID, pdtGroup, 1 as Count from txn3_curr""")
txn5_curr.show()

+--------+--------+-----+
|PARTY_ID|pdtGroup|Count|
+--------+--------+-----+
|       1|      23|    1|
|       8|      24|    1|
+--------+--------+-----+



p58



In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *

data= [['uv', 'sv'], ['am', 'cm'], ["kc", "Bc"], ['lm', "bj"], ["oi", "bh"], ["","us"]]

columns=['newcinsfx1', 'newcinsfx2']

spark = SparkSession.builder.appName('sas2py').getOrCreate()
datamart1=spark.createDataFrame(data=data,schema=columns)
datamart1.createOrReplaceTempView("datamart1")
datamart1.show()



data= [["uv"], ["lk"], ["dk"], ["am"], ['oi'], ["us"]]
columns=['cust_cin']
output=spark.createDataFrame(data=data,schema=columns)
output.createOrReplaceTempView("output")
output.show()

+----------+----------+
|newcinsfx1|newcinsfx2|
+----------+----------+
|        uv|        sv|
|        am|        cm|
|        kc|        Bc|
|        lm|        bj|
|        oi|        bh|
|          |        us|
+----------+----------+

+--------+
|cust_cin|
+--------+
|      uv|
|      lk|
|      dk|
|      am|
|      oi|
|      us|
+--------+



In [ ]:
temp = spark.sql("""select case when newcinsfx1 != "" then newcinsfx1 else newcinsfx2 end as newcinsfx, * from datamart1""")
temp.createOrReplaceTempView("temp")
datamart = spark.sql("""select * from temp where newcinsfx in (select cust_cin from output)""")
datamart.createOrReplaceTempView("datamart")
datamart.show()

+---------+----------+----------+
|newcinsfx|newcinsfx1|newcinsfx2|
+---------+----------+----------+
|       am|        am|        cm|
|       oi|        oi|        bh|
|       us|          |        us|
|       uv|        uv|        sv|
+---------+----------+----------+



p222

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *

data= [[78, 121120, 110821], [16,121020,121121], [4,110920, 121121]]
       
columns=["party_id", "start-dt", "end_dt"]

spark = SparkSession.builder.appName('sas2py').getOrCreate()
dps_retail=spark.createDataFrame(data=data,schema=columns)
#dps_retail.createOrReplaceTempView("dps_retail")
dps_retail.show()

+--------+--------+------+
|party_id|start-dt|end_dt|
+--------+--------+------+
|      78|  121120|110821|
|      16|  121020|121121|
|       4|  110920|121121|
+--------+--------+------+



In [ ]:
def proc_sort(data):
  df = data.sort(["party_id", "start-dt", "end_dt"])
  return df
dps_retail = proc_sort(data=dps_retail)
dps_retail.show()

+--------+--------+------+
|party_id|start-dt|end_dt|
+--------+--------+------+
|       4|  110920|121121|
|      16|  121020|121121|
|      78|  121120|110821|
+--------+--------+------+



p142

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *
spark = SparkSession.builder.appName('sas2py').getOrCreate()

data= [['abcd', 'Deposits', 90], ["klmn", "Insurance", 70], ["bhjk", "Investment", 50]]
columns=["newcinsfx", "product_name", "accuracy"]
CYCLE_DATA=spark.createDataFrame(data=data,schema=columns)
CYCLE_DATA.createOrReplaceTempView("CYCLE_DATA")
CYCLE_DATA.show()

data= [['abcd'], ["klmn"], ["hjkl"], ["okpl"]]
columns=["cust_cin"]
daily_data=spark.createDataFrame(data=data,schema=columns)
daily_data.createOrReplaceTempView("daily_data")
daily_data.show()

+---------+------------+--------+
|newcinsfx|product_name|accuracy|
+---------+------------+--------+
|     abcd|    Deposits|      90|
|     klmn|   Insurance|      70|
|     bhjk|  Investment|      50|
+---------+------------+--------+

+--------+
|cust_cin|
+--------+
|    abcd|
|    klmn|
|    hjkl|
|    okpl|
+--------+



In [ ]:
from pyspark.sql.functions  import concat
spark.sql("""

select distinct newcinsfx, product_name, accuracy, concat(newcinsfx,"-",product_name) as CODE from CYCLE_DATA where (locate(newcinsfx,"STAFF") > 0 or newcinsfx in (select cust_cin from daily_data)) and product_name in ("Deposits","Home Loan","Insurance","Investment") order by code,newcinsfx, product_name, accuracy

""").show()

+---------+------------+--------+--------------+
|newcinsfx|product_name|accuracy|          CODE|
+---------+------------+--------+--------------+
|     abcd|    Deposits|      90| abcd-Deposits|
|     klmn|   Insurance|      70|klmn-Insurance|
+---------+------------+--------+--------------+



In [ ]:
spark.sql("""

create table CUST_EXTFI1 as select distinct newcinsfx, product_name, accuracy, concat(newcinsfx,"-",product_name) as CODE from ${CYCLE_DATA} where (locate(newcinsfx,"STAFF") > 0 or newcinsfx in (select cust_cin from daily_data)) and product_name in ("Deposits","Home Loan","Insurance","Investment") order by code,newcinsfx, product_name, accuracy

""")

p156

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *
spark = SparkSession.builder.appName('sas2py').getOrCreate()

data= [['TRDATA', 'TPC_BLUESKY_REV2'], ["TRDATA", "TPC_BLUESKY_REV2"], ["TDATA", "TPC_BLUESKY_REV2"]]
columns=['libname', 'memname']
dictionary_tables=spark.createDataFrame(data=data,schema=columns)
dictionary_tables.createOrReplaceTempView("dictionary_tables")
dictionary_tables.show()

spark.conf.set("${CDATA_MTH}","TRDATA" )

+-------+----------------+
|libname|         memname|
+-------+----------------+
| TRDATA|TPC_BLUESKY_REV2|
| TRDATA|TPC_BLUESKY_REV2|
|  TDATA|TPC_BLUESKY_REV2|
+-------+----------------+



In [ ]:
spark.conf.set("${CDATA_MTH}","TRDATA" )

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id

result = spark.sql("""

select monotonically_increasing_id() as NUM, concat(libname,".",memname) as table_name, date_format(substr(memname,16,6), '6') as MTH_NUM, 
substr(memname,16,6) as MTH from dictionary_tables where libname="TRDATA" and memname like "%TPC_BLUESKY_REV2%" and 
date_format(substr(memname,16,4), '4')=date_format(substr("${CDATA_MTH}",1,4), '4') - 1 and 
date_format(substr("${CDATA_MTH}",1,6), '6') > date_format(substr(memname,16,6), '6') order by mth desc

""")

result.show()

+---+----------+-------+---+
|NUM|table_name|MTH_NUM|MTH|
+---+----------+-------+---+
+---+----------+-------+---+



In [ ]:
spark.sql("""
create table dictionary as select monotonically_increasing_id() as NUM, concat(libname,".",memname) as table_name, date_format(substr(memname,16,6), '6') as MTH_NUM,
substr(memname,16,6) as MTH from dictionary_tables where libname="TRDATA" and memname like "%TPC_BLUESKY_REV2%" 
and date_format(substr(memname,16,4), '4')=date_format(substr("${CDATA_MTH}",1,4), '4') - 1 and 
date_format(substr("${CDATA_MTH}",1,6), '6') > date_format(substr(memname,16,6), '6') order by mth desc
""")

p254

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *
spark = SparkSession.builder.appName('sas2py').getOrCreate()

data= [['11-01-2020', '12-12-2020', 'sgd', 12], ['22-02-2020', '25-12-2020', 'SGD', 8], ['13-01-2020', '27-10-2020', 'sgd', 2] ]
columns=["start_dt", "end_dt", "base_curr", "curr"]
bip_vcbg_v_d129017_rthis_extn_hist=spark.createDataFrame(data=data,schema=columns)

bip_vcbg_v_d129017_rthis_extn_hist = bip_vcbg_v_d129017_rthis_extn_hist.withColumn('start_dt', to_date(col("start_dt"),"dd-MM-yyyy"))
bip_vcbg_v_d129017_rthis_extn_hist = bip_vcbg_v_d129017_rthis_extn_hist.withColumn('end_dt', to_date(col("end_dt"),"dd-MM-yyyy"))

bip_vcbg_v_d129017_rthis_extn_hist.createOrReplaceTempView("bip_vcbg_v_d129017_rthis_extn_hist")
bip_vcbg_v_d129017_rthis_extn_hist.show()

spark.conf.set("${refresh_date}","10-03-2020" )

+----------+----------+---------+----+
|  start_dt|    end_dt|base_curr|curr|
+----------+----------+---------+----+
|2020-01-11|2020-12-12|      sgd|  12|
|2020-02-22|2020-12-25|      SGD|   8|
|2020-01-13|2020-10-27|      sgd|   2|
+----------+----------+---------+----+



StructType(List(StructField(start_dt,DateType,true),StructField(end_dt,DateType,true),StructField(base_curr,StringType,true),StructField(curr,LongType,true)))

In [ ]:
spark.conf.set("${refresh_date}","10-03-2020" )

In [ ]:
result = spark.sql("""

select * from bip_vcbg_v_d129017_rthis_extn_hist where to_date("${refresh_date}") between start_dt and end_dt and upper(base_curr) = 'SGD'  order by curr, start_dt desc

""")

result.show()

+--------+------+---------+----+
|start_dt|end_dt|base_curr|curr|
+--------+------+---------+----+
+--------+------+---------+----+



Channa

In [ ]:
spark = SparkSession.builder.appName('P116').getOrCreate()
data = [["acvbgf"],["deshyu"],["sdf"]]
columns = ["newcinsfx"]
dset= spark.createDataFrame(data,columns)
dset.createOrReplaceTempView('dset')
dset.show()


data = [["12sd","N","11Jan2022","acv","bgf"],["23fg","N","01Jan9999","des","hyu"],["45hj","N","01Jan9999","jkl","pop"]]
columns = ["party_id","termind","termdate","cin","sfx"]
TCIS= spark.createDataFrame(data,columns)
TCIS.createOrReplaceTempView('TCIS')
TCIS.show()

+---------+
|newcinsfx|
+---------+
|   acvbgf|
|   deshyu|
|      sdf|
+---------+

+--------+-------+---------+---+---+
|party_id|termind| termdate|cin|sfx|
+--------+-------+---------+---+---+
|    12sd|      N|11Jan2022|acv|bgf|
|    23fg|      N|01Jan9999|des|hyu|
|    45hj|      N|01Jan9999|jkl|pop|
+--------+-------+---------+---+---+



In [ ]:
#TCIS=TCIS.withColumn("termdate",to_date(col("termdate"),"dd-MM-yyyy"))

In [ ]:
spark.sql("""

select a.* ,b.party_id as party_id from dset as a left join TCIS as b

on a.newcinsfx = replace(b.cin||b.sfx, ' ', '') where b.party_id != '' and b.termind = 'N' and b.termdate = "01Jan9999"

""").show()

+---------+--------+
|newcinsfx|party_id|
+---------+--------+
|   deshyu|    23fg|
+---------+--------+



184

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *
spark = SparkSession.builder.appName('sas2py').getOrCreate()

data= [['fghd'], ["abcd"]]
columns=["cust_cin"]
TPC_PORTCUBE_QV_REF_DATE=spark.createDataFrame(data=data,schema=columns)
TPC_PORTCUBE_QV_REF_DATE.createOrReplaceTempView("TPC_PORTCUBE_QV_REF_DATE")
TPC_PORTCUBE_QV_REF_DATE.show()

data= [['bhjk', 'ui', 'ij', 'kl', 'mj', 'er', 'tg', 'bi'], ['mklb', 'tg', 'as', 'df', 'gh', 'ty', 'uh', 'th']]  
columns=["newcinsfx", "f_itb", "f_active_90", "f_mbanking", "f_ibmb_active_30", "f_m_active_90", "f_active_30", "f_m_active_30"]
digicust=spark.createDataFrame(data=data,schema=columns)
digicust.createOrReplaceTempView("digicust")
digicust.show()


+--------+
|cust_cin|
+--------+
|    fghd|
|    abcd|
+--------+

+---------+-----+-----------+----------+----------------+-------------+-----------+-------------+
|newcinsfx|f_itb|f_active_90|f_mbanking|f_ibmb_active_30|f_m_active_90|f_active_30|f_m_active_30|
+---------+-----+-----------+----------+----------------+-------------+-----------+-------------+
|     bhjk|   ui|         ij|        kl|              mj|           er|         tg|           bi|
|     mklb|   tg|         as|        df|              gh|           ty|         uh|           th|
+---------+-----+-----------+----------+----------------+-------------+-----------+-------------+



In [ ]:
spark.sql("""
 select a.*, case when (a.cust_cin != e.newcinsfx) then 0 else e.f_itb end as Internet_Banking,
 case when (a.cust_cin != e.newcinsfx) then 0 else e.f_active_90 end as IB_Active, 
 case when (a.cust_cin != e.newcinsfx) then 0 else e.f_mbanking end as Mobile_Banking, 
 case when (a.cust_cin != f.newcinsfx) then 0 else f.f_ibmb_active_30 end as Internet_Mobile_Banking, 
 case when (a.cust_cin != f.newcinsfx) then 0 else f.f_m_active_90 end as MB_Active, 
 case when (e.f_active_90 =1 or f.f_m_active_90 =1) then 1 else 0 end as IB_MB_Active, 
 case when (a.cust_cin != f.newcinsfx) then 0 else f.f_ibmb_active_30 end as IBMB_Active_30, 
 case when (a.cust_cin != f.newcinsfx) then 0 else f.f_m_active_30 end as MB_Active_30, 
 case when (a.cust_cin != f.newcinsfx) then 0 else f.f_active_30 end as IB_Active_30 from TPC_PORTCUBE_QV_REF_DATE a 
 left join digicust e on a.cust_cin = e.newcinsfx left join digicust f on a.cust_cin = f.newcinsfx
 
 """).show()

+--------+----------------+---------+--------------+-----------------------+---------+------------+--------------+------------+------------+
|cust_cin|Internet_Banking|IB_Active|Mobile_Banking|Internet_Mobile_Banking|MB_Active|IB_MB_Active|IBMB_Active_30|MB_Active_30|IB_Active_30|
+--------+----------------+---------+--------------+-----------------------+---------+------------+--------------+------------+------------+
|    abcd|            null|     null|          null|                   null|     null|           0|          null|        null|        null|
|    fghd|            null|     null|          null|                   null|     null|           0|          null|        null|        null|
+--------+----------------+---------+--------------+-----------------------+---------+------------+--------------+------------+------------+



In [ ]:
spark.sql(""" create table TPC_PORTCUBE_QV_${REF_DATE}
 select a.*, case when (a.cust_cin != e.newcinsfx) then 0 else e.f_itb end as Internet_Banking,
 case when (a.cust_cin != e.newcinsfx) then 0 else e.f_active_90 end as IB_Active, 
 case when (a.cust_cin != e.newcinsfx) then 0 else e.f_mbanking end as Mobile_Banking, 
 case when (a.cust_cin != f.newcinsfx) then 0 else f.f_ibmb_active_30 end as Internet_Mobile_Banking, 
 case when (a.cust_cin != f.newcinsfx) then 0 else f.f_m_active_90 end as MB_Active, 
 case when (e.f_active_90 =1 or f.f_m_active_90 =1) then 1 else 0 end as IB_MB_Active, 
 case when (a.cust_cin != f.newcinsfx) then 0 else f.f_ibmb_active_30 end as IBMB_Active_30, 
 case when (a.cust_cin != f.newcinsfx) then 0 else f.f_m_active_30 end as MB_Active_30, 
 case when (a.cust_cin != f.newcinsfx) then 0 else f.f_active_30 end as IB_Active_30 from TPC_PORTCUBE_QV_${REF_DATE} a 
 left join ${digicust} e on a.cust_cin = e.newcinsfx left join ${digicust} f on a.cust_cin = f.newcinsfx
 
 """)

p240

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *
spark = SparkSession.builder.appName('sas2py').getOrCreate()

data= [['TREASURES20gh', 'TRDATA', 'AID'], ['bhTREASURES20', 'TRDATA', 'AID'], ['TREASURES20', 'TRDATA',' CID']]
columns=["memname", "libname", "name"]
dictionary_columns=spark.createDataFrame(data=data,schema=columns)
dictionary_columns.createOrReplaceTempView("dictionary_columns")
dictionary_columns.show()

spark.conf.set("${cdata_mth}",8 )

+-------------+-------+----+
|      memname|libname|name|
+-------------+-------+----+
|TREASURES20gh| TRDATA| AID|
|bhTREASURES20| TRDATA| AID|
|  TREASURES20| TRDATA| CID|
+-------------+-------+----+



In [ ]:
spark.conf.set("${cdata_mth}",8 )

In [ ]:
spark.sql("""
select distinct replace(memname, ' ', '') from dictionary_columns where libname = 'TRDATA' and memname LIKE 'TREASURES20____' and 
name = 'AID' and date_format(substr(memname,length(memname)-5,6), '6') <= "${cdata_mth}" group by libname, memname
having date_format(substr(memname,length(memname)-5,6), '6') = max(date_format(substr(memname,length(memname)-5,6), '6'))
""")

DataFrame[replace(memname,  , ): string]

In [ ]:
spark.sql("""
select distinct replace(memname, ' ', '') from dictionary.columns where libname = 'TRDATA' and memname LIKE 'TREASURES20____' and 
name = 'AID' and date_format(substr(memname,length(memname)-5,6), '6') <= "${cdata_mth}" group by libname, memname
having date_format(substr(memname,length(memname)-5,6), '6') = max(date_format(substr(memname,length(memname)-5,6), '6'))
""")

p2

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *
spark = SparkSession.builder.appName('sas2py').getOrCreate()

data= [['TRDATA', 'TR_ENTITYMART_20_12', 2], ['TRADTA', 'TRENTITYMART20', 1]]
columns=['libname', 'memname', 'nobs']
dictionary_tables=spark.createDataFrame(data=data,schema=columns)
dictionary_tables.createOrReplaceTempView("dictionary_tables")
dictionary_tables.show()

spark.conf.set("${cdata_mth}", 5 )

+-------+-------------------+----+
|libname|            memname|nobs|
+-------+-------------------+----+
| TRDATA|TR_ENTITYMART_20_12|   2|
| TRADTA|     TRENTITYMART20|   1|
+-------+-------------------+----+



In [ ]:
spark.conf.set("${cdata_mth}", 5 )

In [ ]:
spark.sql("""

select distinct replace(libname||"."||memname, ' ', ''),
date_format(last_day(add_months(date_format(replace(substr(memname,length(memname)-5,6), ' ', ''), 'yyyMM'), 0)), 'ddMMMyyy') , 
:MONTHEND_DATE from dictionary_tables where libname = 'TRDATA' and memname LIKE 'TR_ENTITYMART_20____' and nobs > 0 
and date_format(substr(memname,length(memname)-5,6), '6') <= ${cdata_mth} group by libname 
having date_format(substr(memname,length(memname)-5,6), '6') = max(date_format(substr(memname,length(memname)-5,6), '6'))

""")

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



ParseException: ignored

187

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *
spark = SparkSession.builder.appName('sas2py').getOrCreate()

data= [['INV', 'T_AVQPB_CUSTMAST_20_er', 2], ['INV', 'TRENTITYMART20', 1]]
columns=['libname', 'memname', 'nobs']
dictionary_tables=spark.createDataFrame(data=data,schema=columns)
dictionary_tables.createOrReplaceTempView("dictionary_tables")
dictionary_tables.show()

spark.conf.set("${cdata_mth}", 5 )

+-------+--------------------+----+
|libname|             memname|nobs|
+-------+--------------------+----+
|    INV|T_AVQPB_CUSTMAST_...|   2|
|    INV|      TRENTITYMART20|   1|
+-------+--------------------+----+



In [ ]:
spark.sql("""

select distinct replace(memname, ' ', '')  from dictionary_tables where libname = 'INV' and 
memname LIKE 'T_AVQPB_CUSTMAST_20____' and nobs > 0 and date_format(substr(memname,length(memname)-5,6), '6') <= ${cdata_mth} 
group by libname having date_format(substr(memname,length(memname)-5,6), '6') = max(date_format(substr(memname,length(memname)-5,6), '6'))

""")

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



ParseException: ignored

In [ ]:
#216 Jyotsna

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *

data= [

       [43, '19jun2020', '16may2022'],

       [62, '18Jan2020', '20Jun2022'],

       [20, '20nov2020', '14Feb2022'],

       [78, '13dec2020', '29dec2022'],

       [55, '28May2023', '15May2022']

       ]

columns=['BP_S_NUMBER', 'start_dt', 'OPEN_DATE']
spark = SparkSession.builder.appName('sas2py').getOrCreate()
dps1=spark.createDataFrame(data=data,schema=columns)
dps1.show()

+-----------+---------+---------+
|BP_S_NUMBER| start_dt|OPEN_DATE|
+-----------+---------+---------+
|         43|19jun2020|16may2022|
|         62|18Jan2020|20Jun2022|
|         20|20nov2020|14Feb2022|
|         78|13dec2020|29dec2022|
|         55|28May2023|15May2022|
+-----------+---------+---------+



In [ ]:
def proc_sort(data):

  result = data.sort("BP_S_NUMBER","start_dt", "OPEN_DATE")
  return result

result = proc_sort(data=dps1)
result.show()


+-----------+---------+---------+
|BP_S_NUMBER| start_dt|OPEN_DATE|
+-----------+---------+---------+
|         20|20nov2020|14Feb2022|
|         43|19jun2020|16may2022|
|         55|28May2023|15May2022|
|         62|18Jan2020|20Jun2022|
|         78|13dec2020|29dec2022|
+-----------+---------+---------+



In [ ]:
# 230 

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('app').getOrCreate()

data=[[2,3],[1,2],[1,8],[4,5],[5,5],[6,6], [2,3], [1,8]]
columns=['id', 'newcinsfx']

incepted_rp_ul = spark.createDataFrame(data=data,schema=columns)
incepted_rp_ul.show()

+---+---------+
| id|newcinsfx|
+---+---------+
|  2|        3|
|  1|        2|
|  1|        8|
|  4|        5|
|  5|        5|
|  6|        6|
|  2|        3|
|  1|        8|
+---+---------+



In [ ]:
def proc_data(data):
  result=data.sort("newcinsfx")
  result.dropDuplicates()
  return result

incepted_rp_ul=proc_data(data=incepted_rp_ul)
incepted_rp_ul.show(truncate=False)

+---+---------+
|id |newcinsfx|
+---+---------+
|1  |2        |
|2  |3        |
|4  |5        |
|5  |5        |
|6  |6        |
|1  |8        |
+---+---------+



In [ ]:
# 244

In [ ]:
spark = SparkSession.builder.appName('P116').getOrCreate()
data = [[1,2],[3,1],[1,1], [4,1],[7,1], [4,1]]

columns = ['newcinsfx', 'cef_qtd']
CEF2= spark.createDataFrame(data,columns)

CEF2.show()
CEF2.createOrReplaceTempView('CEF2')


+---------+-------+
|newcinsfx|cef_qtd|
+---------+-------+
|        1|      2|
|        3|      1|
|        1|      1|
|        4|      1|
|        7|      1|
|        4|      1|
+---------+-------+



In [ ]:
cef_qtd1 = spark.sql("""select distinct newcinsfx,cef_qtd,1 as cef_q1 from CEF2 where cef_qtd=1""")
cef_qtd1.show()

+---------+-------+------+
|newcinsfx|cef_qtd|cef_q1|
+---------+-------+------+
|        1|      1|     1|
|        3|      1|     1|
|        7|      1|     1|
|        4|      1|     1|
+---------+-------+------+



In [ ]:
#202

In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('app').getOrCreate()
data = [[1,2,3],[2,3,4],[5,8,3],[9,5,4]] 
columns = ["cnt_engage_rm_mtd", "latest_engagement_mtd", "newcinsfx"]
tr_engagement_rm_mtd_all = spark.createDataFrame(data=data, schema=columns)
tr_engagement_rm_mtd_all.createOrReplaceTempView("tr_engagement_rm_mtd_all")
tr_engagement_rm_mtd_all.show()

+-----------------+---------------------+---------+
|cnt_engage_rm_mtd|latest_engagement_mtd|newcinsfx|
+-----------------+---------------------+---------+
|                1|                    2|        3|
|                2|                    3|        4|
|                5|                    8|        3|
|                9|                    5|        4|
+-----------------+---------------------+---------+



In [ ]:
tr_engagement_rm_mtd = spark.sql("""
select newcinsfx, sum(cnt_engage_rm_mtd) as cnt_engage_rm_mtd, 
max(latest_engagement_mtd) as latest_engagement_mtd  from tr_engagement_rm_mtd_all group by newcinsfx
""")

tr_engagement_rm_mtd.show()

+---------+-----------------+---------------------+
|newcinsfx|cnt_engage_rm_mtd|latest_engagement_mtd|
+---------+-----------------+---------------------+
|        3|                6|                    8|
|        4|               11|                    5|
+---------+-----------------+---------------------+



In [ ]:
#258

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *

spark = SparkSession.builder.appName('P116').getOrCreate()
data = [["23-01-2020","13-12-2020"]]
columns = ["start_dt","end_dt"]

bip_vcbg_v_d161025_asset_hist= spark.createDataFrame(data,columns)
bip_vcbg_v_d161025_asset_hist.createOrReplaceTempView('bip_vcbg_v_d161025_asset_hist')
bip_vcbg_v_d161025_asset_hist.show()

spark.conf.set("refresh_date",'2020-01-23')

+----------+----------+
|  start_dt|    end_dt|
+----------+----------+
|23-01-2020|13-12-2020|
+----------+----------+



In [ ]:
asset = spark.sql("""select * from bip_vcbg_v_d161025_asset_hist where to_date('${refresh_date}') 
between to_date(start_dt,'dd-MM-yyy') and to_date(end_dt,'dd-MM-yyy')""")

asset.show()

+----------+----------+
|  start_dt|    end_dt|
+----------+----------+
|23-01-2020|13-12-2020|
+----------+----------+



In [ ]:
#p118

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *
spark = SparkSession.builder.appName('P118').getOrCreate()

data = [[1,3],[1,2]]
column=["PC_CODE_NAME","rm_1bank_id"]

_output_refresh_date_e= spark.createDataFrame(data=data, schema=column)
_output_refresh_date_e.createOrReplaceTempView('_output_refresh_date_e')
_output_refresh_date_e.show()


data = [[3,6],[1,2]]
column=["PC_CODE_NAME","wm_1bank_id"]

chk1bank_id_update1= spark.createDataFrame(data=data, schema=column)
chk1bank_id_update1.createOrReplaceTempView('chk1bank_id_update1')
chk1bank_id_update1.show()

+------------+-----------+
|PC_CODE_NAME|rm_1bank_id|
+------------+-----------+
|           1|          3|
|           1|          2|
+------------+-----------+

+------------+-----------+
|PC_CODE_NAME|wm_1bank_id|
+------------+-----------+
|           3|          6|
|           1|          2|
+------------+-----------+



In [ ]:
_output_refresh_date_f = spark.sql("""select a.*,b.PC_CODE_NAME as PC_CODE_NAME_new from _output_refresh_date_e as a 
left join chk1bank_id_update1 as b on a.rm_1bank_id=b.wm_1bank_id""")

_output_refresh_date_f .show()

+------------+-----------+----------------+
|PC_CODE_NAME|rm_1bank_id|PC_CODE_NAME_new|
+------------+-----------+----------------+
|           1|          2|               1|
|           1|          3|            null|
+------------+-----------+----------------+



In [ ]:
#p180

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.getOrCreate()
data=[["1","2","3"],["11","15","9"],["1","9","2"]]
inv_sales= spark.createDataFrame(data,['newcinsfx','f_oet','f_ofi'])
inv_sales.createOrReplaceTempView("inv_sales")
inv_sales=spark.sql("""SELECT newcinsfx, sum(f_oet) as f_oet, sum(f_ofi) as f_ofi FROM inv_sales GROUP BY 1""")

data=[["1","5"],["15","4"],["14","6"]]
inv_fx= spark.createDataFrame(data,['newcinsfx','f_ofx'])
inv_fx.createOrReplaceTempView("inv_fx")
inv_fx=spark.sql("""SELECT newcinsfx, sum(f_ofx) as f_ofx FROM inv_fx GROUP BY 1""")

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *
spark = SparkSession.builder.appName('sastopyspark').getOrCreate()

data = [[12,34,87], [11,35,89], [12,32,88],[15,43,78],[11,53,90],[15,87,34]]
column=['newcinsfx', 'f_oet', 'f_ofi']

inv_sales= spark.createDataFrame(data=data, schema=column)
inv_sales.createOrReplaceTempView('inv_sales')
inv_sales.show()


data = [[12,67],[13,98],[14,65],[15,99],[12,13],[13,56],[14,78]]
column=['newcinsfx', 'f_ofx']

inv_fx= spark.createDataFrame(data=data, schema=column)
inv_fx.createOrReplaceTempView('inv_fx')
inv_fx.show()

+---------+-----+-----+
|newcinsfx|f_oet|f_ofi|
+---------+-----+-----+
|       12|   34|   87|
|       11|   35|   89|
|       12|   32|   88|
|       15|   43|   78|
|       11|   53|   90|
|       15|   87|   34|
+---------+-----+-----+

+---------+-----+
|newcinsfx|f_ofx|
+---------+-----+
|       12|   67|
|       13|   98|
|       14|   65|
|       15|   99|
|       12|   13|
|       13|   56|
|       14|   78|
+---------+-----+



In [ ]:
inv_sales = spark.sql("SELECT newcinsfx, sum(f_oet) as f_oet,  sum(f_ofi) as f_ofi FROM inv_sales GROUP BY 1")
inv_sales.show()
inv_fx = spark.sql("SELECT newcinsfx, sum(f_ofx) as f_ofx FROM inv_fx GROUP BY 1")
inv_fx.show()

+---------+-----+-----+
|newcinsfx|f_oet|f_ofi|
+---------+-----+-----+
|       12|   66|  175|
|       11|   88|  179|
|       15|  130|  112|
+---------+-----+-----+

+---------+-----+
|newcinsfx|f_ofx|
+---------+-----+
|       12|   80|
|       13|  154|
|       14|  143|
|       15|   99|
+---------+-----+



In [ ]:
#p211

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *
spark = SparkSession.builder.appName('sastopyspark').getOrCreate()

data = [[1, 2, 'abcdefgh', 'product1', '7585', 12042022, 23042022, 10042022, 30042022],
        [2, 3, 'ijklmnop', 'product2', '7581', 14032022, 25032022, 10032022, 28032022],
        [4, 6, 'abgcvhij',  'product7', '7589', 14032022, 25032022, 10032022, 28032022]]

column= ['bp_id', 'cont_id', 'sort_alpha', 'name1', 'extn_cont_type_id', 'open_date', 'close_date', 'start_dt', 'end_dt']

bip_vcbg_v_d161012_cont_hist= spark.createDataFrame(data=data, schema=column)
bip_vcbg_v_d161012_cont_hist.createOrReplaceTempView('bip_vcbg_v_d161012_cont_hist')
bip_vcbg_v_d161012_cont_hist.show()

spark.conf.set("refresh_date", 15042022)

+-----+-------+----------+--------+-----------------+---------+----------+--------+--------+
|bp_id|cont_id|sort_alpha|   name1|extn_cont_type_id|open_date|close_date|start_dt|  end_dt|
+-----+-------+----------+--------+-----------------+---------+----------+--------+--------+
|    1|      2|  abcdefgh|product1|             7585| 12042022|  23042022|10042022|30042022|
|    2|      3|  ijklmnop|product2|             7581| 14032022|  25032022|10032022|28032022|
|    4|      6|  abgcvhij|product7|             7589| 14032022|  25032022|10032022|28032022|
+-----+-------+----------+--------+-----------------+---------+----------+--------+--------+



In [ ]:
spark.sql("""
select bp_id, cont_id, sort_alpha, substr(sort_alpha,1,8) as bp_s_number,name1, extn_cont_type_id,
open_date, close_date, start_dt, end_dt from bip_vcbg_v_d161012_cont_hist where extn_cont_type_id in ('7585','7589', '7592') 
and open_date <= '${refresh_date}' and (close_date >= '${refresh_date}' or close_date is null) and end_dt >= '${refresh_date}'
""").show()

+-----+-------+----------+-----------+--------+-----------------+---------+----------+--------+--------+
|bp_id|cont_id|sort_alpha|bp_s_number|   name1|extn_cont_type_id|open_date|close_date|start_dt|  end_dt|
+-----+-------+----------+-----------+--------+-----------------+---------+----------+--------+--------+
|    1|      2|  abcdefgh|   abcdefgh|product1|             7585| 12042022|  23042022|10042022|30042022|
|    4|      6|  abgcvhij|   abgcvhij|product7|             7589| 14032022|  25032022|10032022|28032022|
+-----+-------+----------+-----------+--------+-----------------+---------+----------+--------+--------+



In [ ]:
data=[["1","2","3",'7589','10-09-1998','13-02-2025','13-04-2026','4','c','109','11-04-2026','10-04-2026'],
      ["11","15","5",'7585','12-08-1996','10-05-2045','19-01-2998','1','b','107','18-04-2026','10-04-2026'],
      ["1","9","2","7289",'11-03-2022','12-03-2023','11-04-2022','2','a','101','10-04-2026','10-04-2026']]
columns = ['bp_id','cont_id','sort_alpha','extn_cont_type_id','open_date','close_date','end_dt','bp_s_number',
           'name1','extn_cont_type_id1','refresh_date','start_dt']

bip_vcbg_v_d161012_cont_hist= spark.createDataFrame(data,columns)
bip_vcbg_v_d161012_cont_hist.createOrReplaceTempView("bip_vcbg_v_d161012_cont_hist")
refresh_date="12-12-2022"
spark.conf.set("refresh_date",refresh_date)
bip_vcbg_v_d161012_cont_hist.show()

+-----+-------+----------+-----------------+----------+----------+----------+-----------+-----+------------------+------------+----------+
|bp_id|cont_id|sort_alpha|extn_cont_type_id| open_date|close_date|    end_dt|bp_s_number|name1|extn_cont_type_id1|refresh_date|  start_dt|
+-----+-------+----------+-----------------+----------+----------+----------+-----------+-----+------------------+------------+----------+
|    1|      2|         3|             7589|10-09-1998|13-02-2025|13-04-2026|          4|    c|               109|  11-04-2026|10-04-2026|
|   11|     15|         5|             7585|12-08-1996|10-05-2045|19-01-2998|          1|    b|               107|  18-04-2026|10-04-2026|
|    1|      9|         2|             7289|11-03-2022|12-03-2023|11-04-2022|          2|    a|               101|  10-04-2026|10-04-2026|
+-----+-------+----------+-----------------+----------+----------+----------+-----------+-----+------------------+------------+----------+



In [ ]:
spark.sql("""
select bp_id, cont_id, sort_alpha, substr(sort_alpha,1,8) as bp_s_number,name1, extn_cont_type_id,
open_date, close_date, start_dt, end_dt from bip_vcbg_v_d161012_cont_hist where extn_cont_type_id in ('7585','7589', '7592') 
and open_date <= '${refresh_date}' and (close_date >= '${refresh_date}' or close_date is null) and end_dt >= '${refresh_date}'
""").show()

+-----+-------+----------+-----------+-----+-----------------+----------+----------+----------+----------+
|bp_id|cont_id|sort_alpha|bp_s_number|name1|extn_cont_type_id| open_date|close_date|  start_dt|    end_dt|
+-----+-------+----------+-----------+-----+-----------------+----------+----------+----------+----------+
|    1|      2|         3|          3|    c|             7589|10-09-1998|13-02-2025|10-04-2026|13-04-2026|
+-----+-------+----------+-----------+-----+-----------------+----------+----------+----------+----------+



In [ ]:
dpmdps = spark.sql("""select bp_id, cont_id, sort_alpha, substr(sort_alpha,1,8) as bp_s_number, name1, extn_cont_type_id , 
open_date, close_date, start_dt, end_dt from bip_vcbg_v_d161012_cont_hist where extn_cont_type_id in ('7585','7589', '7592') 
and to_date(open_date,'dd-mm-yyyy') <= to_date("${refresh_date}",'dd-mm-yyyy') and 
((to_date(close_date,'dd-mm-yyyy') >= to_date("${refresh_date}",'dd-mm-yyyy') or 
to_date(close_date,'dd-mm-yyyy') is null) and to_date(end_dt,'dd-mm-yyyy') >= to_date("${refresh_date}",'dd-mm-yyyy'))""")

dpmdps.show()

+-----+-------+----------+-----------+-----+-----------------+----------+----------+----------+----------+
|bp_id|cont_id|sort_alpha|bp_s_number|name1|extn_cont_type_id| open_date|close_date|  start_dt|    end_dt|
+-----+-------+----------+-----------+-----+-----------------+----------+----------+----------+----------+
|    1|      2|         3|          3|    c|             7589|10-09-1998|13-02-2025|10-04-2026|13-04-2026|
|   11|     15|         5|          5|    b|             7585|12-08-1996|10-05-2045|10-04-2026|19-01-2998|
+-----+-------+----------+-----------+-----+-----------------+----------+----------+----------+----------+



## p151

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *
spark = SparkSession.builder.appName('sastopyspark').getOrCreate()

#base

data = [
        
        ['er', 'ahah', 'abc', 'Upendra', 1, 1, 'yes', 1, 'yes', 'no', 1, 'uv67', 'tr56',
         'yes', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'yes',' yes', 'yes', 1, 1, 
         1, 'run65', 'upen', 'ranchi', 25, 'bokaro', 'single', 'M8', 'ind8', '2020', 'Salaried',
         'uyay', 1, 1]
        
       ]

column= ['CUST_ID', 'CALENDAR_DATE', 'newcinsfx', 'CUST_FULL_NAME', 'ASIA_TRSUR_IND', 'AVALOQ_HOLDING_IND',
         'DEPOSIT_IND', 'EML_IND', 'BABYSIT_IND', 'BUSINESS_CAUTIOUS_IND', 'INTERNET_BNKNG_IND', 'MRTG_INd',
         'OET_IND', 'SECURED_LOAN_IND', 'SECURITIES_IND', 'TBE_MAIN_IND', 'TBE_SUPPL_IND', 'UNSECURED_LOAN_IND',
         'SJ_ACCT_1', 'SJ_ACCT_2', 'SJ_ACCT_3', 'SJ_ACCT_4', 'SJ_ACCT_5', 'SJ_ACCT_6', 'UNSUB_MKTG_FAX_IND', 'UNSUB_MKTG_SMS_IND',
         'UNSUB_TELEMKTG_IND', 'CAMPAIGN_RUN_DT', 'CUST_NATIONALITY_CNTRY_NAME', 'CUST_RESIDENCY_CNTRY_NAME',
         'AGE', 'DISTRICT', 'MARITAL_STATUS_TYPE', 'CUST_GENDER_TYPE_CODE', 'BUSINESS_DIVISION_ID', 'REL_YEAR_WITH_TR',
         'OCCUPATION_TYPE', 'DNC_VALIDITY_DATE', 'INVST_IND', 'TRSRY_IND']

base= spark.createDataFrame(data=data, schema=column)
base.createOrReplaceTempView('base')
base.show()


#trm_cube_qv_final_refresh_date

data = [
        
        ['abc', 1, 1, 1, 1, 1, 1, 'no', 1, 1, 1, 1, 1, 1, 70000, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 'yes']
  
       ]

column= ['newcinsfx', 'f_p6m_txn_cli', 'f_eln', 'f_p6m_txn_eln', 'f_mrtl', 'f_ttsd',
         'f_wealth_holdings', 'MISMTH_WH', 'f_bond', 'f_cli', 'f_sn', 'f_ut', 'f_investment',
         'f_treasury', 'income', 'f_insurance', 'f_anz', 'f_inslife_RP', 'f_inslife_SP', 
         'f_universal_life', 'f_gi', 'f_pdpa_email', 'f_pdpa_dm', 'f_pdpa_fax', 'f_pdpa_tm',
         'f_pdpa_sms', 'f_cust_new']

trm_cube_qv_final_refresh_date= spark.createDataFrame(data=data, schema=column)
trm_cube_qv_final_refresh_date.createOrReplaceTempView('trm_cube_qv_final_refresh_date')
trm_cube_qv_final_refresh_date.show()

#output_refresh_date

data = [
        
        ['abc', 'yes', 'ind', 'bnk', 'july', 'monday', 'zebra',' acv', 'ds', 'wma', 'ram',' no', 'no', 212,
         'mshra', 'yes', 'bvbv', 'RamKumar', 'ramesh', 'InsightData']
  
       ]

column= ['newcinsfx', 'STAFF_IND', 'AI_GROUP', 'RM_1BANK_ID', 'BIRTH_Month',
         'Birth_day', 'PC_Code_NAME', 'PC_Code', 'SUBTEAM', 'WM_1BANK_ID', 'AUM_LOCAL_CBG',
         'SC_DEP_AUM_BLUESKY', 'SC_INV_AUM_BLUESKY', 'ENTITY_AUM', 'RM_CBGBA_TEAM_NAME', 'MISMTH_QUAL',
         'AI_EXPIRY_DT', 'RM_FULL_NAME', 'RM_LAN_ID', 'SoftTag_IND']

output_refresh_date= spark.createDataFrame(data=data, schema=column)
output_refresh_date.createOrReplaceTempView('output_refresh_date')
output_refresh_date.show()


#cycle_data

data = [
        
        ['abc', 'star', 'yes', 'ind', 'ind56', 3, 2, 12, 'yes', 18, 'yes', 13, 
         'yes', 'no', 'no',' no', 'no', 'yes', 1, 'yes', 1, 'yes', 1, 'yes', 1, 
         'no', 'zer', 'art', 'rtyu', 'uhuh', 'astg', 'reyg', 'yes', 'no', 'yes']
  
       ]

column= ['newcinsfx', 'STAR_LS', 'TOP_LIFESTYLE', 'TOP_LOC_REGN', 'TOP_LOC_SUBZ',
         'CHILD_CNT', 'CHILD_AGE_GROUP', 'CHILD1_AGE', 'CHILD1_AGE_ACC',
         'CHILD2_AGE', 'CHILD2_AGE_ACC', 'CHILD3_AGE', 'CHILD3_AGE_ACC', 'CHILD4_AGE',
         'CHILD4_AGE_ACC', 'CHILD5_AGE', 'CHILD5_AGE_ACC', 'TOP_OVS_LOC', 'OB_DEP_IND',
         'OB_DEP_ACC', 'OB_INS_IND', 'OB_INS_ACC', 'OB_HLN_IND', 'OB_HLN_ACC', 'OB_INV_IND',
         'OB_INV_ACC', 'PREF_FC_DEP1', 'PREF_FC_DEP2', 'PREF_FC_DEP3', 'PREF_FC_CC1', 'PREF_FC_CC2',
         'PREF_FC_CC3', 'IBG_12', 'IBG_34', 'TOP_INB_CHANNEL']

cycle_data= spark.createDataFrame(data=data, schema=column)
cycle_data.createOrReplaceTempView('cycle_data')
cycle_data.show()


#tr_an_riskprofile_new


data = [['yes', 'a']]
column= ['client_risk_profile', 'cin_fna_done_on']

tr_an_riskprofile_new= spark.createDataFrame(data=data, schema=column)
tr_an_riskprofile_new.createOrReplaceTempView('tr_an_riskprofile_new')
tr_an_riskprofile_new.show()


#wlth_pen

data = [['er', 'yes']]
column= ['party_id', 'f_wlth_pen']

wlth_pen= spark.createDataFrame(data=data, schema=column)
wlth_pen.createOrReplaceTempView('wlth_pen')
wlth_pen.show()

#tciscust

data = [['a', 'bc']]
column= ['CIN', 'SFX']

tciscust= spark.createDataFrame(data=data, schema=column)
tciscust.createOrReplaceTempView('tciscust')
tciscust.show()

spark.conf.set("MONTHEND_DATE", '2014-12-31')

+-------+-------------+---------+--------------+--------------+------------------+-----------+-------+-----------+---------------------+------------------+--------+-------+----------------+--------------+------------+-------------+------------------+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+---------------+---------------------------+-------------------------+---+--------+-------------------+---------------------+--------------------+----------------+---------------+-----------------+---------+---------+
|CUST_ID|CALENDAR_DATE|newcinsfx|CUST_FULL_NAME|ASIA_TRSUR_IND|AVALOQ_HOLDING_IND|DEPOSIT_IND|EML_IND|BABYSIT_IND|BUSINESS_CAUTIOUS_IND|INTERNET_BNKNG_IND|MRTG_INd|OET_IND|SECURED_LOAN_IND|SECURITIES_IND|TBE_MAIN_IND|TBE_SUPPL_IND|UNSECURED_LOAN_IND|SJ_ACCT_1|SJ_ACCT_2|SJ_ACCT_3|SJ_ACCT_4|SJ_ACCT_5|SJ_ACCT_6|UNSUB_MKTG_FAX_IND|UNSUB_MKTG_SMS_IND|UNSUB_TELEMKTG_IND|CAMPAIGN_RUN_DT|CUST_NATIONALITY_CNTRY_NAME|CUST

In [ ]:
TR_PORTCUBE_QV1 = spark.sql("""

select a.CUST_ID , a.CALENDAR_DATE , a.newcinsfx as CUST_CIN , case when c.STAFF_IND = "Y" then "STAFF" 
when c.STAFF_IND != "Y" then a.CUST_FULL_NAME when a.CUST_FULL_NAME = "" then "N/A" else "N/A" end as CUST_FULL_NAME , 
case when a.ASIA_TRSUR_IND=1 then "Y" else "N" end as ASIA_TRSUR_IND , 
case when a.AVALOQ_HOLDING_IND="1" then "Y" else "N" end as AVALOQ_HOLDING_IND , 
case when b.f_p6m_txn_cli=1 then "Y" else "N" end as CLI_TXN_6_MNTH_IND , 
case when a.DEPOSIT_IND="1" then "Y" else "N" end as DEPOSIT_IND , 
case when b.f_eln=1 then "Y" else "N" end as AVALOQ_ELN_IND , 
case when b.f_p6m_txn_eln=1 then "Y" else "N" end as ELN_TXN_6_MNTH_IND , 
case when a.EML_IND=1 then "Y" else "N" end as EML_IND , case when a.BABYSIT_IND="1" then "Y" else "N" end as BABYSIT_IND , 
case when a.BUSINESS_CAUTIOUS_IND="1" then "Y" else "N" end as BUSINESS_CAUTIOUS_IND, 
case when a.INTERNET_BNKNG_IND="1" then "Y" else "N" end as INTERNET_BNKNG_IND, case when b.f_mrtl = 1 then 'Y' else "N" end as LVAGE_ACCT_IND, 
case when a.MRTG_IND="1" then "Y" else "N" end as MRTG_IND, case when a.OET_IND="1" then "Y" else "N" end as OET_IND, 
case when a.SECURED_LOAN_IND="1" then "Y" else "N" end as SECURED_LOAN_IND, 
case when a.SECURITIES_IND="1" then "Y" else "N" end as SECURITIES_IND, 
case when b.f_ttsd=1 then "Y" else "N" end as STRUCTURED_DEPOSIT_IND, 
case when a.TBE_MAIN_IND="1" then "Y" else "N" end as TBE_MAIN_IND, 
case when a.TBE_SUPPL_IND="1" then "Y" else "N" end as TBE_SUPPL_IND, 
case when a.UNSECURED_LOAN_IND="1" then "Y" else "N" end as UNSECURED_LOAN_IND, 
case when b.f_wealth_holdings=1 then "Y" else "N" end as WEALTH_HOLDING_IND, b.MISMTH_WH, 
case when a.SJ_ACCT_1 != "" then a.SJ_ACCT_1 else "N/A" end as SJ_ACCT_1, 
case when a.SJ_ACCT_2 != "" then a.SJ_ACCT_2 else "N/A" end as SJ_ACCT_2, 
case when a.SJ_ACCT_3 != "" then a.SJ_ACCT_3 else "N/A" end as SJ_ACCT_3, 
case when a.SJ_ACCT_4 != "" then a.SJ_ACCT_4 else "N/A" end as SJ_ACCT_4, 
case when a.SJ_ACCT_5 != "" then a.SJ_ACCT_5 else "N/A" end as SJ_ACCT_5, 
case when a.SJ_ACCT_6 != "" then a.SJ_ACCT_6 else "N/A" end as SJ_ACCT_6, 
case when a.UNSUB_MKTG_FAX_IND=1 then "Y" else "N" end as UNSUB_MKTG_FAX_IND, 
case when a.UNSUB_MKTG_SMS_IND=1 then "Y" else "N" end as UNSUB_MKTG_SMS_IND, 
case when a.UNSUB_TELEMKTG_IND=1 then "Y" else "N" end as UNSUB_TELEMKTG_IND, a.CAMPAIGN_RUN_DT, 
case when a.CUST_NATIONALITY_CNTRY_NAME != "" then a.CUST_NATIONALITY_CNTRY_NAME else "N/A" end as CUST_NATIONALITY_CNTRY_NAME, 
case when a.CUST_RESIDENCY_CNTRY_NAME != "" then a.CUST_RESIDENCY_CNTRY_NAME else "N/A" end as CUST_RESIDENCY_CNTRY_NAME, a.AGE, 
case when a.AGE >= 0 and a.AGE < 18 then '01. 0 - 18' when a.AGE >= 18 and a.AGE < 25 then '02. 18 - 25' 
when a.AGE >= 25 and a.AGE < 30 then '03. 25 - 30' when a.AGE >= 30 and a.AGE < 40 then '04. 30 - 40' 
when a.AGE >= 40 and a.AGE < 50 then '05. 40 - 50' when a.AGE >= 50 and a.AGE < 60 then '06. 50 - 60' 
when a.AGE >= 60 and a.AGE < 65 then '07. 60 - 65' when a.AGE >= 65 and a.AGE < 70 then '08. 65 - 70' 
when a.AGE >= 70 and a.AGE < 80 then '09. 70 - 80' when a.AGE >= 80 then '10. >=80' else '00. N/A' end as Age_Group, 
case when a.DISTRICT != "" then a.DISTRICT else "N/A" end as DISTRICT, case when a.MARITAL_STATUS_TYPE='MRD' then 'Married' 
when a.MARITAL_STATUS_TYPE='SNGL' then 'Single' when a.MARITAL_STATUS_TYPE='SPRTD' then 'Separated' else 'N/A' end as MARITAL_STATUS_TYPE, 
case when a.CUST_GENDER_TYPE_CODE='F' then 'Female' when a.CUST_GENDER_TYPE_CODE='M' then 'Male' else 'N/A' end as CUST_GENDER_TYPE_CODE, 
case when a.BUSINESS_DIVISION_ID != "" then a.BUSINESS_DIVISION_ID else "N/A" end as BUSINESS_DIVISION_ID, a.REL_YEAR_WITH_TR, 
case when a.REL_YEAR_WITH_TR >= 0 and a.REL_YEAR_WITH_TR < 1 then '01. 0 - <1 yrs' 
when a.REL_YEAR_WITH_TR >= 1 and a.REL_YEAR_WITH_TR < 2 then '02. 1 - <2 yrs' 
when a.REL_YEAR_WITH_TR >= 2 and a.REL_YEAR_WITH_TR < 3 then '03. 2 - <3 yrs' 
when a.REL_YEAR_WITH_TR >= 3 and a.REL_YEAR_WITH_TR < 5 then '04. 3 - <5 yrs' 
when a.REL_YEAR_WITH_TR >= 5 and a.REL_YEAR_WITH_TR < 6 then '05. 5 - <6 yrs' 
when a.REL_YEAR_WITH_TR >= 6 and a.REL_YEAR_WITH_TR < 7 then '06. 6 - <7 yrs' 
when a.REL_YEAR_WITH_TR >= 7 and a.REL_YEAR_WITH_TR < 9 then '07. 7 - <9 yrs' 
when a.REL_YEAR_WITH_TR >= 9 then '08. 9 + yrs' else '00. N/A' end as Treasure_Relation_Tenure, 
case when f.client_risk_profile != "" then f.client_risk_profile else "N/A" end as RISK_PROFILE_TYPE_CODE, 
case when a.OCCUPATION_TYPE != "" then a.OCCUPATION_TYPE else "N/A" end as OCCUPATION_TYPE, 
case when a.OCCUPATION_TYPE='ADMIN' then 'Administrative' when a.OCCUPATION_TYPE='AGRIC' then 'Agricultural' 
when a.OCCUPATION_TYPE='AUTOHIGHRSKBUS' then 'High Risk Business' when a.OCCUPATION_TYPE='CLERIC' then 'Clerical' 
when a.OCCUPATION_TYPE='HOUSEWIFE' then 'Housewife' when a.OCCUPATION_TYPE='NATL_SVCMN' then 'National Serviceman' 
when a.OCCUPATION_TYPE='NONWRKR' then 'Non-Worker' when a.OCCUPATION_TYPE='NOT_AVLBL' then 'N/A' 
when a.OCCUPATION_TYPE='OTHR' then 'Others' when a.OCCUPATION_TYPE='PRDN' then 'Production' 
when a.OCCUPATION_TYPE='PRFSNL' then 'Professional' when a.OCCUPATION_TYPE='RETIREE' then 'Retiree' 
when a.OCCUPATION_TYPE='SALES' then 'Sales' when a.OCCUPATION_TYPE='SRVC' then 'Service' 
when a.OCCUPATION_TYPE='STUDENT' then 'Student' when a.OCCUPATION_TYPE='SLF_EMPLYD' then 'Self-Employed' else "N/A" end as Occupation, 
a.DNC_VALIDITY_DATE, case when b.f_bond=1 then "Y" else "N" end as BOND_IND, case when b.f_cli=1 then "Y" else "N" end as CLI_IND, 
case when a.INVST_IND=1 then "Y" else "N" end as INVST_IND, case when a.TRSRY_IND=1 then "Y" else "N" end as TRSRY_IND, 
case when b.f_sn=1 then "Y" else "N" end as STRUCTURED_NOTE_IND, case when b.f_ut=1 then "Y" else "N" end as UNIT_TRUST_IND, 
case when b.f_investment=1 then "Y" else "N" end as INVST_HOLDING_IND, 
case when b.f_treasury=1 then "Y" else "N" end as TRSRY_HOLDING_IND, b.income as INCOME_TWELVE_MONTH_AVG, 
case when b.income IS NULL then 'N/A' when b.income < 0 then '01. < 0' when b.income >= 0 and b.income < 2500 
then '02. 0 to 2.5K' when b.income >= 2500 and b.income < 5000 then '03. 2.5K to 5K' when b.income >= 5000 and b.income < 7500 
then '04. 5K to 7.5K' when b.income >= 7500 and b.income < 10000 then '05. 7.5K to 10K' when b.income >= 10000 and b.income < 15000 
then '06. 10K to 15K' when b.income >= 15000 and b.income < 20000 then '07. 15K to 20K' when b.income >= 20000 and b.income < 30000 
then '08. 20K to 30K' when b.income >= 30000 then'09. >= 30K' else 'N/A' end as Income_Group, "${MONTHEND_DATE}" as CALENDAR_MONTH_END_DATE ,
case when c.AI_GROUP = 'AI' then 'Y' else "N" end as AI_DERIVED_IND, 
case when c.RM_1BANK_ID != "" then upper(c.RM_1BANK_ID) else "N/A" end as RM_1BANK_ID, 
case when c.Birth_Month != "" then c.Birth_Month else "N/A" end as BDAY_M, c.Birth_day as BDAY, 
case when c.PC_CODE_NAME != "" then c.PC_CODE_NAME else "N/A" end as PC_CODE_NAME, 
case when c.PC_CODE != "" then c.PC_CODE else "N/A" end as PC_CODE, case when c.SUBTEAM != "" then c.SUBTEAM else "N/A" end as SUBTEAM, 
case when c.WM_1BANK_ID != "" then c.WM_1BANK_ID else "N/A" end as WM_1BANK_ID, 
c.AUM_LOCAL_CBG, c.SC_DEP_AUM_BLUESKY, c.SC_INV_AUM_BLUESKY, c.ENTITY_AUM, case when c.ENTITY_AUM <= 0 then'<= 0' 
when c.ENTITY_AUM > 0 and c.ENTITY_AUM < 80000 then '0 to 80K' when c.ENTITY_AUM >= 80000 and c.ENTITY_AUM < 200000 
then '80K to 200K' when c.ENTITY_AUM >= 200000 and c.ENTITY_AUM < 350000 then '200K to 350K' 
when c.ENTITY_AUM >= 350000 and c.ENTITY_AUM < 500000 then '350K to 500K' when c.ENTITY_AUM >= 500000 
and c.ENTITY_AUM < 750000 then '500K to 750K' when c.ENTITY_AUM >= 750000 and c.ENTITY_AUM < 1000000 then '750K to 1Mil' 
when c.ENTITY_AUM >= 1000000 and c.ENTITY_AUM < 1500000 then '1Mil to 1.5Mil' when c.ENTITY_AUM >= 1500000 and c.ENTITY_AUM < 3000000 
then '1.5Mil to 3Mil' when c.ENTITY_AUM >= 3000000 and c.ENTITY_AUM < 5000000 then '3Mil to 5Mil' when c.ENTITY_AUM > 5000000 
then '>= 5Mil' else 'N/A' end as AUM_TIER_DETAIL, case when c.ENTITY_AUM < 200000 then '<200K' when c.ENTITY_AUM >= 200000 and 
c.ENTITY_AUM < 350000 then '200K to 350K' when c.ENTITY_AUM >= 350000 and c.ENTITY_AUM < 1000000 then '350K to 1Mil' 
when c.ENTITY_AUM >= 1000000 then '>=1Mil' else 'N/A' end as AUM_TIER, case when locate(c.RM_CBGBA_TEAM_NAME,"TPC") > 0 and 
c.ENTITY_AUM >= 1500000 then 'Y' when locate(c.RM_CBGBA_TEAM_NAME,"TPC") = 0 and c.ENTITY_AUM >= 200000 then 'Y' else 'N' 
end as Qualified_Indicator, c.MISMTH_QUAL, c.AI_EXPIRY_DT, case when c.AI_GROUP != "" then c.AI_GROUP else "N/A" 
end as AI_GROUP, case when c.RM_CBGBA_TEAM_NAME != "" then c.RM_CBGBA_TEAM_NAME else "N/A" end as RM_CBGBA_TEAM_NAME, case 
when c. RM_FULL_NAME != "" then c.RM_FULL_NAME else "N/A" end as RM_FULL_NAME, case when c.RM_LAN_ID != "" then trim(c.RM_LAN_ID) else 'N/A' 
end as RM_LAN_ID, case when c.STAFF_IND = "Y" then "Y" else 'N' end as STAFF_IND, 
case when b.f_insurance=1 then "Y" else "N" end as f_insurance, 
case when b.f_inslife_RP=1 then "Y" else "N" end as f_inslife_payor_RP, 
case when b.f_inslife_SP=1 then "Y" else "N" end as f_inslife_payor_SP, 
case when b.f_universal_life=1 then "Y" else "N" end as f_universal_life, 
case when b.f_gi=1 then "Y" else "N" end as f_GI, case when b.f_pdpa_email=0 then "Y" else "N" end as PDPA_EMAIL_IND, 
case when b.f_pdpa_dm=0 then "Y" else "N" end as PDPA_DM_IND, case when b.f_pdpa_fax=0 then "Y" else "N" end as PDPA_FAX_IND, 
case when b.f_pdpa_tm=0 then "Y" else "N" end as PDPA_TM_IND, case when b.f_pdpa_sms=0 then "Y" else "N" end as PDPA_SMS_IND, 
case when b.f_cust_new != "" then b.f_cust_new else "N/A" end as CUST_NEW_IND, 
case when e.TOP_LIFESTYLE != "" then e.TOP_LIFESTYLE else "N/A" end as TOP_LIFESTYLE, e.STAR_LS, 
case when e.TOP_LOC_REGN != "" then e.TOP_LOC_REGN else "N/A" end as TOP_LOC_REGN, 
case when e.TOP_LOC_SUBZ != "" then e.TOP_LOC_SUBZ else "N/A" end as TOP_LOC_SUBZ, e.CHILD_CNT, 
case when e.CHILD_AGE_GROUP != "" then e.CHILD_AGE_GROUP else "N/A" end as CHILD_AGE_GROUP, e.CHILD1_AGE, 
case when e.CHILD1_AGE_ACC != "" then e.CHILD1_AGE_ACC else "" end as CHILD1_AGE_ACC, e.CHILD2_AGE, 
case when e.CHILD2_AGE_ACC != "" then e.CHILD2_AGE_ACC else "" end as CHILD2_AGE_ACC, e.CHILD3_AGE,
case when e.CHILD3_AGE_ACC != "" then e.CHILD3_AGE_ACC else "" end as CHILD3_AGE_ACC, e.CHILD4_AGE, 
case when e.CHILD4_AGE_ACC != "" then e.CHILD4_AGE_ACC else "" end as CHILD4_AGE_ACC, e.CHILD5_AGE, 
case when e.CHILD5_AGE_ACC != "" then e.CHILD5_AGE_ACC else "" end as CHILD5_AGE_ACC, 
case when e.TOP_OVS_LOC != "" then e.TOP_OVS_LOC else "N/A" end as TOP_OVS_LOC, 
case when e.OB_DEP_IND=1 then "Y" else "N" end as OB_DEP_IND, 
case when e.OB_DEP_ACC != "" then e.OB_DEP_ACC else "" end as OB_DEP_ACC, 
case when e.OB_INS_IND=1 then "Y" else "N" end as OB_INS_IND, case when e.OB_INS_ACC != "" then e.OB_INS_ACC else "" end as OB_INS_ACC, 
case when e.OB_HLN_IND=1 then "Y" else "N" end as OB_HLN_IND, case when e.OB_HLN_ACC != "" then e.OB_HLN_ACC else "" end as OB_HLN_ACC, 
case when e.OB_INV_IND=1 then "Y" else "N" end as OB_INV_IND, case when e.OB_INV_ACC != "" then e.OB_INV_ACC else "" end as OB_INV_ACC, 
case when e.PREF_FC_DEP1 != "" then e.PREF_FC_DEP1 else "N/A" end as PREF_FC_DEP1, 
case when e.PREF_FC_DEP2 != "" then e.PREF_FC_DEP2 else "N/A" end as PREF_FC_DEP2, 
case when e.PREF_FC_DEP3 != "" then e.PREF_FC_DEP3 else "N/A" end as PREF_FC_DEP3, 
case when e.PREF_FC_CC1 != "" then e.PREF_FC_CC1 else "N/A" end as PREF_FC_CC1, 
case when e.PREF_FC_CC2 != "" then e.PREF_FC_CC2 else "N/A" end as PREF_FC_CC2, 
case when e.PREF_FC_CC3 != "" then e.PREF_FC_CC3 else "N/A" end as PREF_FC_CC3, 
case when e.IBG_12 != "" then e.IBG_12 else "N" end as IBG_12, case when e.IBG_34 != "" then e.IBG_34 else "N" end as IBG_34, 
case when e.TOP_INB_CHANNEL != "" then e.TOP_INB_CHANNEL else "N/A" end as TOP_INB_CHANNEL, 
case when b.f_anz = 1 then 'Y' else 'N' end as F_ANZ, case when g.f_wlth_pen != '' then g.f_wlth_pen else 'No Wealth Penetration' 
end as F_WLTH_PEN, c.SoftTag_Ind from base as a left join trm_cube_qv_final_refresh_date as b on a.newcinsfx=b.newcinsfx 
left join output_refresh_date as c on a.newcinsfx=c.newcinsfx left join cycle_data as e on a.newcinsfx=e.newcinsfx 
left join tr_an_riskprofile_new as f on substr(a.newcinsfx,1,length(a.newcinsfx)-2)=f.cin_fna_done_on 
left join wlth_pen as g on a.cust_id = g.party_id where substr(c.RM_LAN_ID,1,4) != 'PVBK' and 
a.newcinsfx in (select CONCAT(CIN,SFX) from tciscust)
""")

TR_PORTCUBE_QV1.show()

+-------+-------------+--------+--------------+--------------+------------------+------------------+-----------+--------------+------------------+-------+-----------+---------------------+------------------+--------------+--------+-------+----------------+--------------+----------------------+------------+-------------+------------------+------------------+---------+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+---------------+---------------------------+-------------------------+---+-----------+--------+-------------------+---------------------+--------------------+----------------+------------------------+----------------------+---------------+----------+-----------------+--------+-------+---------+---------+-------------------+--------------+-----------------+-----------------+-----------------------+------------+-----------------------+--------------+-----------+------+------+------------+-------+-------+-------

## p151 another approach (dynamic variabel)

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *
spark = SparkSession.builder.appName('sastopyspark').getOrCreate()

#base
data = [['er', 'ahah', 'abc', 'Upendra', 1, 1, 'yes', 1, 'yes', 'no', 1, 'uv67', 'tr56',
         'yes', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'yes',' yes', 'yes', 1, 1, 
         1, 'run65', 'upen', 'ranchi', 25, 'bokaro', 'single', 'M8', 'ind8', '2020', 'Salaried',
         'uyay', 1, 1]]

column= ['CUST_ID', 'CALENDAR_DATE', 'newcinsfx', 'CUST_FULL_NAME', 'ASIA_TRSUR_IND', 'AVALOQ_HOLDING_IND',
         'DEPOSIT_IND', 'EML_IND', 'BABYSIT_IND', 'BUSINESS_CAUTIOUS_IND', 'INTERNET_BNKNG_IND', 'MRTG_INd',
         'OET_IND', 'SECURED_LOAN_IND', 'SECURITIES_IND', 'TBE_MAIN_IND', 'TBE_SUPPL_IND', 'UNSECURED_LOAN_IND',
         'SJ_ACCT_1', 'SJ_ACCT_2', 'SJ_ACCT_3', 'SJ_ACCT_4', 'SJ_ACCT_5', 'SJ_ACCT_6', 'UNSUB_MKTG_FAX_IND', 'UNSUB_MKTG_SMS_IND',
         'UNSUB_TELEMKTG_IND', 'CAMPAIGN_RUN_DT', 'CUST_NATIONALITY_CNTRY_NAME', 'CUST_RESIDENCY_CNTRY_NAME',
         'AGE', 'DISTRICT', 'MARITAL_STATUS_TYPE', 'CUST_GENDER_TYPE_CODE', 'BUSINESS_DIVISION_ID', 'REL_YEAR_WITH_TR',
         'OCCUPATION_TYPE', 'DNC_VALIDITY_DATE', 'INVST_IND', 'TRSRY_IND']

base= spark.createDataFrame(data=data, schema=column)
base.createOrReplaceTempView('base')
base.show()

#trm_cube_qv_final_refresh_date
data = [['abc', 1, 1, 1, 1, 1, 1, 'no', 1, 1, 1, 1, 1, 1, 70000, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 'yes']]

column= ['newcinsfx', 'f_p6m_txn_cli', 'f_eln', 'f_p6m_txn_eln', 'f_mrtl', 'f_ttsd',
         'f_wealth_holdings', 'MISMTH_WH', 'f_bond', 'f_cli', 'f_sn', 'f_ut', 'f_investment',
         'f_treasury', 'income', 'f_insurance', 'f_anz', 'f_inslife_RP', 'f_inslife_SP', 
         'f_universal_life', 'f_gi', 'f_pdpa_email', 'f_pdpa_dm', 'f_pdpa_fax', 'f_pdpa_tm',
         'f_pdpa_sms', 'f_cust_new']

spark.conf.set("refresh_date",10)
trm_cube_qv_final_refresh_date= spark.createDataFrame(data=data, schema=column)
trm_cube_qv_final_refresh_date.show()
trm_cube_qv_final_refresh_date.createOrReplaceTempView('trm_cube_qv_final_10')


#output_refresh_date

data = [['abc', 'yes', 'ind', 'bnk', 'july', 'monday', 'zebra',' acv', 'ds', 'wma', 'ram',' no', 'no', 212,
         'mshra', 'yes', 'bvbv', 'RamKumar', 'ramesh', 'InsightData']]

column= ['newcinsfx', 'STAFF_IND', 'AI_GROUP', 'RM_1BANK_ID', 'BIRTH_Month',
         'Birth_day', 'PC_Code_NAME', 'PC_Code', 'SUBTEAM', 'WM_1BANK_ID', 'AUM_LOCAL_CBG',
         'SC_DEP_AUM_BLUESKY', 'SC_INV_AUM_BLUESKY', 'ENTITY_AUM', 'RM_CBGBA_TEAM_NAME', 'MISMTH_QUAL',
         'AI_EXPIRY_DT', 'RM_FULL_NAME', 'RM_LAN_ID', 'SoftTag_IND']

spark.conf.set("output","output")
output_refresh_date= spark.createDataFrame(data=data, schema=column)
output_refresh_date.show()
output_refresh_date.createOrReplaceTempView('output_10')



#cycle_data

data = [['abc', 'star', 'yes', 'ind', 'ind56', 3, 2, 12, 'yes', 18, 'yes', 13, 
         'yes', 'no', 'no',' no', 'no', 'yes', 1, 'yes', 1, 'yes', 1, 'yes', 1, 
         'no', 'zer', 'art', 'rtyu', 'uhuh', 'astg', 'reyg', 'yes', 'no', 'yes']]

column= ['newcinsfx', 'STAR_LS', 'TOP_LIFESTYLE', 'TOP_LOC_REGN', 'TOP_LOC_SUBZ',
         'CHILD_CNT', 'CHILD_AGE_GROUP', 'CHILD1_AGE', 'CHILD1_AGE_ACC',
         'CHILD2_AGE', 'CHILD2_AGE_ACC', 'CHILD3_AGE', 'CHILD3_AGE_ACC', 'CHILD4_AGE',
         'CHILD4_AGE_ACC', 'CHILD5_AGE', 'CHILD5_AGE_ACC', 'TOP_OVS_LOC', 'OB_DEP_IND',
         'OB_DEP_ACC', 'OB_INS_IND', 'OB_INS_ACC', 'OB_HLN_IND', 'OB_HLN_ACC', 'OB_INV_IND',
         'OB_INV_ACC', 'PREF_FC_DEP1', 'PREF_FC_DEP2', 'PREF_FC_DEP3', 'PREF_FC_CC1', 'PREF_FC_CC2',
         'PREF_FC_CC3', 'IBG_12', 'IBG_34', 'TOP_INB_CHANNEL']

cycle_data= spark.createDataFrame(data=data, schema=column)
cycle_data.createOrReplaceTempView('cycle_data')
cycle_data.show()


#tr_an_riskprofile_new
data = [['yes', 'a']]
column= ['client_risk_profile', 'cin_fna_done_on']

spark.conf.set("riskprofile_new",20)
tr_an_riskprofile_new= spark.createDataFrame(data=data, schema=column)
tr_an_riskprofile_new.show()
tr_an_riskprofile_new.createOrReplaceTempView('tr_an_20')

#wlth_pen
data = [['er', 'yes']]
column= ['party_id', 'f_wlth_pen']

wlth_pen= spark.createDataFrame(data=data, schema=column)
wlth_pen.createOrReplaceTempView('wlth_pen')
wlth_pen.show()

#tciscust
data = [['a', 'bc']]
column= ['CIN', 'SFX']

tciscust= spark.createDataFrame(data=data, schema=column)
tciscust.createOrReplaceTempView('tciscust')
tciscust.show()

spark.conf.set("MONTHEND_DATE", '2014-12-31')

+-------+-------------+---------+--------------+--------------+------------------+-----------+-------+-----------+---------------------+------------------+--------+-------+----------------+--------------+------------+-------------+------------------+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+---------------+---------------------------+-------------------------+---+--------+-------------------+---------------------+--------------------+----------------+---------------+-----------------+---------+---------+
|CUST_ID|CALENDAR_DATE|newcinsfx|CUST_FULL_NAME|ASIA_TRSUR_IND|AVALOQ_HOLDING_IND|DEPOSIT_IND|EML_IND|BABYSIT_IND|BUSINESS_CAUTIOUS_IND|INTERNET_BNKNG_IND|MRTG_INd|OET_IND|SECURED_LOAN_IND|SECURITIES_IND|TBE_MAIN_IND|TBE_SUPPL_IND|UNSECURED_LOAN_IND|SJ_ACCT_1|SJ_ACCT_2|SJ_ACCT_3|SJ_ACCT_4|SJ_ACCT_5|SJ_ACCT_6|UNSUB_MKTG_FAX_IND|UNSUB_MKTG_SMS_IND|UNSUB_TELEMKTG_IND|CAMPAIGN_RUN_DT|CUST_NATIONALITY_CNTRY_NAME|CUST

In [ ]:
TR_PORTCUBE_QV1 = spark.sql("""

select a.CUST_ID , a.CALENDAR_DATE , a.newcinsfx as CUST_CIN , case when c.STAFF_IND = "Y" then "STAFF" 
when c.STAFF_IND != "Y" then a.CUST_FULL_NAME when a.CUST_FULL_NAME = "" then "N/A" else "N/A" end as CUST_FULL_NAME , 
case when a.ASIA_TRSUR_IND=1 then "Y" else "N" end as ASIA_TRSUR_IND , 
case when a.AVALOQ_HOLDING_IND="1" then "Y" else "N" end as AVALOQ_HOLDING_IND , 
case when b.f_p6m_txn_cli=1 then "Y" else "N" end as CLI_TXN_6_MNTH_IND , 
case when a.DEPOSIT_IND="1" then "Y" else "N" end as DEPOSIT_IND , 
case when b.f_eln=1 then "Y" else "N" end as AVALOQ_ELN_IND , 
case when b.f_p6m_txn_eln=1 then "Y" else "N" end as ELN_TXN_6_MNTH_IND , 
case when a.EML_IND=1 then "Y" else "N" end as EML_IND , case when a.BABYSIT_IND="1" then "Y" else "N" end as BABYSIT_IND , 
case when a.BUSINESS_CAUTIOUS_IND="1" then "Y" else "N" end as BUSINESS_CAUTIOUS_IND, 
case when a.INTERNET_BNKNG_IND="1" then "Y" else "N" end as INTERNET_BNKNG_IND, case when b.f_mrtl = 1 then 'Y' else "N" end as LVAGE_ACCT_IND, 
case when a.MRTG_IND="1" then "Y" else "N" end as MRTG_IND, case when a.OET_IND="1" then "Y" else "N" end as OET_IND, 
case when a.SECURED_LOAN_IND="1" then "Y" else "N" end as SECURED_LOAN_IND, 
case when a.SECURITIES_IND="1" then "Y" else "N" end as SECURITIES_IND, 
case when b.f_ttsd=1 then "Y" else "N" end as STRUCTURED_DEPOSIT_IND, 
case when a.TBE_MAIN_IND="1" then "Y" else "N" end as TBE_MAIN_IND, 
case when a.TBE_SUPPL_IND="1" then "Y" else "N" end as TBE_SUPPL_IND, 
case when a.UNSECURED_LOAN_IND="1" then "Y" else "N" end as UNSECURED_LOAN_IND, 
case when b.f_wealth_holdings=1 then "Y" else "N" end as WEALTH_HOLDING_IND, b.MISMTH_WH, 
case when a.SJ_ACCT_1 != "" then a.SJ_ACCT_1 else "N/A" end as SJ_ACCT_1, 
case when a.SJ_ACCT_2 != "" then a.SJ_ACCT_2 else "N/A" end as SJ_ACCT_2, 
case when a.SJ_ACCT_3 != "" then a.SJ_ACCT_3 else "N/A" end as SJ_ACCT_3, 
case when a.SJ_ACCT_4 != "" then a.SJ_ACCT_4 else "N/A" end as SJ_ACCT_4, 
case when a.SJ_ACCT_5 != "" then a.SJ_ACCT_5 else "N/A" end as SJ_ACCT_5, 
case when a.SJ_ACCT_6 != "" then a.SJ_ACCT_6 else "N/A" end as SJ_ACCT_6, 
case when a.UNSUB_MKTG_FAX_IND=1 then "Y" else "N" end as UNSUB_MKTG_FAX_IND,
case when a.UNSUB_MKTG_SMS_IND=1 then "Y" else "N" end as UNSUB_MKTG_SMS_IND, 
case when a.UNSUB_TELEMKTG_IND=1 then "Y" else "N" end as UNSUB_TELEMKTG_IND, a.CAMPAIGN_RUN_DT, 
case when a.CUST_NATIONALITY_CNTRY_NAME != "" then a.CUST_NATIONALITY_CNTRY_NAME else "N/A" end as CUST_NATIONALITY_CNTRY_NAME, 
case when a.CUST_RESIDENCY_CNTRY_NAME != "" then a.CUST_RESIDENCY_CNTRY_NAME else "N/A" end as CUST_RESIDENCY_CNTRY_NAME, a.AGE, 
case when a.AGE >= 0 and a.AGE < 18 then '01. 0 - 18' when a.AGE >= 18 and a.AGE < 25 then '02. 18 - 25' 
when a.AGE >= 25 and a.AGE < 30 then '03. 25 - 30' when a.AGE >= 30 and a.AGE < 40 then '04. 30 - 40' 
when a.AGE >= 40 and a.AGE < 50 then '05. 40 - 50' when a.AGE >= 50 and a.AGE < 60 then '06. 50 - 60' 
when a.AGE >= 60 and a.AGE < 65 then '07. 60 - 65' when a.AGE >= 65 and a.AGE < 70 then '08. 65 - 70' 
when a.AGE >= 70 and a.AGE < 80 then '09. 70 - 80' when a.AGE >= 80 then '10. >=80' else '00. N/A' end as Age_Group, 
case when a.DISTRICT != "" then a.DISTRICT else "N/A" end as DISTRICT, case when a.MARITAL_STATUS_TYPE='MRD' then 'Married' 
when a.MARITAL_STATUS_TYPE='SNGL' then 'Single' when a.MARITAL_STATUS_TYPE='SPRTD' then 'Separated' else 'N/A' end as MARITAL_STATUS_TYPE, 
case when a.CUST_GENDER_TYPE_CODE='F' then 'Female' when a.CUST_GENDER_TYPE_CODE='M' then 'Male' else 'N/A' end as CUST_GENDER_TYPE_CODE, 
case when a.BUSINESS_DIVISION_ID != "" then a.BUSINESS_DIVISION_ID else "N/A" end as BUSINESS_DIVISION_ID, a.REL_YEAR_WITH_TR, 
case when a.REL_YEAR_WITH_TR >= 0 and a.REL_YEAR_WITH_TR < 1 then '01. 0 - <1 yrs' 
when a.REL_YEAR_WITH_TR >= 1 and a.REL_YEAR_WITH_TR < 2 then '02. 1 - <2 yrs' 
when a.REL_YEAR_WITH_TR >= 2 and a.REL_YEAR_WITH_TR < 3 then '03. 2 - <3 yrs' 
when a.REL_YEAR_WITH_TR >= 3 and a.REL_YEAR_WITH_TR < 5 then '04. 3 - <5 yrs' 
when a.REL_YEAR_WITH_TR >= 5 and a.REL_YEAR_WITH_TR < 6 then '05. 5 - <6 yrs' 
when a.REL_YEAR_WITH_TR >= 6 and a.REL_YEAR_WITH_TR < 7 then '06. 6 - <7 yrs' 
when a.REL_YEAR_WITH_TR >= 7 and a.REL_YEAR_WITH_TR < 9 then '07. 7 - <9 yrs' 
when a.REL_YEAR_WITH_TR >= 9 then '08. 9 + yrs' else '00. N/A' end as Treasure_Relation_Tenure, 
case when f.client_risk_profile != "" then f.client_risk_profile else "N/A" end as RISK_PROFILE_TYPE_CODE, 
case when a.OCCUPATION_TYPE != "" then a.OCCUPATION_TYPE else "N/A" end as OCCUPATION_TYPE, 
case when a.OCCUPATION_TYPE='ADMIN' then 'Administrative' when a.OCCUPATION_TYPE='AGRIC' then 'Agricultural' 
when a.OCCUPATION_TYPE='AUTOHIGHRSKBUS' then 'High Risk Business' when a.OCCUPATION_TYPE='CLERIC' then 'Clerical' 
when a.OCCUPATION_TYPE='HOUSEWIFE' then 'Housewife' when a.OCCUPATION_TYPE='NATL_SVCMN' then 'National Serviceman' 
when a.OCCUPATION_TYPE='NONWRKR' then 'Non-Worker' when a.OCCUPATION_TYPE='NOT_AVLBL' then 'N/A' 
when a.OCCUPATION_TYPE='OTHR' then 'Others' when a.OCCUPATION_TYPE='PRDN' then 'Production' 
when a.OCCUPATION_TYPE='PRFSNL' then 'Professional' when a.OCCUPATION_TYPE='RETIREE' then 'Retiree' 
when a.OCCUPATION_TYPE='SALES' then 'Sales' when a.OCCUPATION_TYPE='SRVC' then 'Service' 
when a.OCCUPATION_TYPE='STUDENT' then 'Student' when a.OCCUPATION_TYPE='SLF_EMPLYD' then 'Self-Employed' else "N/A" end as Occupation, 
a.DNC_VALIDITY_DATE, case when b.f_bond=1 then "Y" else "N" end as BOND_IND, case when b.f_cli=1 then "Y" else "N" end as CLI_IND, 
case when a.INVST_IND=1 then "Y" else "N" end as INVST_IND, case when a.TRSRY_IND=1 then "Y" else "N" end as TRSRY_IND, 
case when b.f_sn=1 then "Y" else "N" end as STRUCTURED_NOTE_IND, case when b.f_ut=1 then "Y" else "N" end as UNIT_TRUST_IND, 
case when b.f_investment=1 then "Y" else "N" end as INVST_HOLDING_IND, 
case when b.f_treasury=1 then "Y" else "N" end as TRSRY_HOLDING_IND, b.income as INCOME_TWELVE_MONTH_AVG, 
case when b.income IS NULL then 'N/A' when b.income < 0 then '01. < 0' when b.income >= 0 and b.income < 2500 
then '02. 0 to 2.5K' when b.income >= 2500 and b.income < 5000 then '03. 2.5K to 5K' when b.income >= 5000 and b.income < 7500 
then '04. 5K to 7.5K' when b.income >= 7500 and b.income < 10000 then '05. 7.5K to 10K' when b.income >= 10000 and b.income < 15000 
then '06. 10K to 15K' when b.income >= 15000 and b.income < 20000 then '07. 15K to 20K' when b.income >= 20000 and b.income < 30000 
then '08. 20K to 30K' when b.income >= 30000 then'09. >= 30K' else 'N/A' end as Income_Group, "${MONTHEND_DATE}" as CALENDAR_MONTH_END_DATE ,
case when c.AI_GROUP = 'AI' then 'Y' else "N" end as AI_DERIVED_IND, 
case when c.RM_1BANK_ID != "" then upper(c.RM_1BANK_ID) else "N/A" end as RM_1BANK_ID, 
case when c.Birth_Month != "" then c.Birth_Month else "N/A" end as BDAY_M, c.Birth_day as BDAY, 
case when c.PC_CODE_NAME != "" then c.PC_CODE_NAME else "N/A" end as PC_CODE_NAME, 
case when c.PC_CODE != "" then c.PC_CODE else "N/A" end as PC_CODE, case when c.SUBTEAM != "" then c.SUBTEAM else "N/A" end as SUBTEAM, 
case when c.WM_1BANK_ID != "" then c.WM_1BANK_ID else "N/A" end as WM_1BANK_ID, 
c.AUM_LOCAL_CBG, c.SC_DEP_AUM_BLUESKY, c.SC_INV_AUM_BLUESKY, c.ENTITY_AUM, case when c.ENTITY_AUM <= 0 then'<= 0' 
when c.ENTITY_AUM > 0 and c.ENTITY_AUM < 80000 then '0 to 80K' when c.ENTITY_AUM >= 80000 and c.ENTITY_AUM < 200000 
then '80K to 200K' when c.ENTITY_AUM >= 200000 and c.ENTITY_AUM < 350000 then '200K to 350K' 
when c.ENTITY_AUM >= 350000 and c.ENTITY_AUM < 500000 then '350K to 500K' when c.ENTITY_AUM >= 500000 
and c.ENTITY_AUM < 750000 then '500K to 750K' when c.ENTITY_AUM >= 750000 and c.ENTITY_AUM < 1000000 then '750K to 1Mil' 
when c.ENTITY_AUM >= 1000000 and c.ENTITY_AUM < 1500000 then '1Mil to 1.5Mil' when c.ENTITY_AUM >= 1500000 and c.ENTITY_AUM < 3000000 
then '1.5Mil to 3Mil' when c.ENTITY_AUM >= 3000000 and c.ENTITY_AUM < 5000000 then '3Mil to 5Mil' when c.ENTITY_AUM > 5000000 
then '>= 5Mil' else 'N/A' end as AUM_TIER_DETAIL, case when c.ENTITY_AUM < 200000 then '<200K' when c.ENTITY_AUM >= 200000 and 
c.ENTITY_AUM < 350000 then '200K to 350K' when c.ENTITY_AUM >= 350000 and c.ENTITY_AUM < 1000000 then '350K to 1Mil' 
when c.ENTITY_AUM >= 1000000 then '>=1Mil' else 'N/A' end as AUM_TIER, case when locate(c.RM_CBGBA_TEAM_NAME,"TPC") > 0 and 
c.ENTITY_AUM >= 1500000 then 'Y' when locate(c.RM_CBGBA_TEAM_NAME,"TPC") = 0 and c.ENTITY_AUM >= 200000 then 'Y' else 'N' 
end as Qualified_Indicator, c.MISMTH_QUAL, c.AI_EXPIRY_DT, case when c.AI_GROUP != "" then c.AI_GROUP else "N/A" 
end as AI_GROUP, case when c.RM_CBGBA_TEAM_NAME != "" then c.RM_CBGBA_TEAM_NAME else "N/A" end as RM_CBGBA_TEAM_NAME, case 
when c. RM_FULL_NAME != "" then c.RM_FULL_NAME else "N/A" end as RM_FULL_NAME, case when c.RM_LAN_ID != "" then trim(c.RM_LAN_ID) else 'N/A' 
end as RM_LAN_ID, case when c.STAFF_IND = "Y" then "Y" else 'N' end as STAFF_IND, 
case when b.f_insurance=1 then "Y" else "N" end as f_insurance, 
case when b.f_inslife_RP=1 then "Y" else "N" end as f_inslife_payor_RP, 
case when b.f_inslife_SP=1 then "Y" else "N" end as f_inslife_payor_SP, 
case when b.f_universal_life=1 then "Y" else "N" end as f_universal_life, 
case when b.f_gi=1 then "Y" else "N" end as f_GI, case when b.f_pdpa_email=0 then "Y" else "N" end as PDPA_EMAIL_IND, 
case when b.f_pdpa_dm=0 then "Y" else "N" end as PDPA_DM_IND, case when b.f_pdpa_fax=0 then "Y" else "N" end as PDPA_FAX_IND, 
case when b.f_pdpa_tm=0 then "Y" else "N" end as PDPA_TM_IND, case when b.f_pdpa_sms=0 then "Y" else "N" end as PDPA_SMS_IND, 
case when b.f_cust_new != "" then b.f_cust_new else "N/A" end as CUST_NEW_IND, 
case when e.TOP_LIFESTYLE != "" then e.TOP_LIFESTYLE else "N/A" end as TOP_LIFESTYLE, e.STAR_LS, 
case when e.TOP_LOC_REGN != "" then e.TOP_LOC_REGN else "N/A" end as TOP_LOC_REGN, 
case when e.TOP_LOC_SUBZ != "" then e.TOP_LOC_SUBZ else "N/A" end as TOP_LOC_SUBZ, e.CHILD_CNT, 
case when e.CHILD_AGE_GROUP != "" then e.CHILD_AGE_GROUP else "N/A" end as CHILD_AGE_GROUP, e.CHILD1_AGE, 
case when e.CHILD1_AGE_ACC != "" then e.CHILD1_AGE_ACC else "" end as CHILD1_AGE_ACC, e.CHILD2_AGE, 
case when e.CHILD2_AGE_ACC != "" then e.CHILD2_AGE_ACC else "" end as CHILD2_AGE_ACC, e.CHILD3_AGE,
case when e.CHILD3_AGE_ACC != "" then e.CHILD3_AGE_ACC else "" end as CHILD3_AGE_ACC, e.CHILD4_AGE, 
case when e.CHILD4_AGE_ACC != "" then e.CHILD4_AGE_ACC else "" end as CHILD4_AGE_ACC, e.CHILD5_AGE, 
case when e.CHILD5_AGE_ACC != "" then e.CHILD5_AGE_ACC else "" end as CHILD5_AGE_ACC, 
case when e.TOP_OVS_LOC != "" then e.TOP_OVS_LOC else "N/A" end as TOP_OVS_LOC, 
case when e.OB_DEP_IND=1 then "Y" else "N" end as OB_DEP_IND, 
case when e.OB_DEP_ACC != "" then e.OB_DEP_ACC else "" end as OB_DEP_ACC, 
case when e.OB_INS_IND=1 then "Y" else "N" end as OB_INS_IND, case when e.OB_INS_ACC != "" then e.OB_INS_ACC else "" end as OB_INS_ACC, 
case when e.OB_HLN_IND=1 then "Y" else "N" end as OB_HLN_IND, case when e.OB_HLN_ACC != "" then e.OB_HLN_ACC else "" end as OB_HLN_ACC, 
case when e.OB_INV_IND=1 then "Y" else "N" end as OB_INV_IND, case when e.OB_INV_ACC != "" then e.OB_INV_ACC else "" end as OB_INV_ACC, 
case when e.PREF_FC_DEP1 != "" then e.PREF_FC_DEP1 else "N/A" end as PREF_FC_DEP1, 
case when e.PREF_FC_DEP2 != "" then e.PREF_FC_DEP2 else "N/A" end as PREF_FC_DEP2, 
case when e.PREF_FC_DEP3 != "" then e.PREF_FC_DEP3 else "N/A" end as PREF_FC_DEP3, 
case when e.PREF_FC_CC1 != "" then e.PREF_FC_CC1 else "N/A" end as PREF_FC_CC1, 
case when e.PREF_FC_CC2 != "" then e.PREF_FC_CC2 else "N/A" end as PREF_FC_CC2, 
case when e.PREF_FC_CC3 != "" then e.PREF_FC_CC3 else "N/A" end as PREF_FC_CC3, 
case when e.IBG_12 != "" then e.IBG_12 else "N" end as IBG_12, case when e.IBG_34 != "" then e.IBG_34 else "N" end as IBG_34, 
case when e.TOP_INB_CHANNEL != "" then e.TOP_INB_CHANNEL else "N/A" end as TOP_INB_CHANNEL, 
case when b.f_anz = 1 then 'Y' else 'N' end as F_ANZ, case when g.f_wlth_pen != '' then g.f_wlth_pen else 'No Wealth Penetration' 
end as F_WLTH_PEN, c.SoftTag_Ind from base as a 
left join trm_cube_qv_final_${refresh_date} as b on a.newcinsfx=b.newcinsfx 
left join ${output}_${refresh_date} as c on a.newcinsfx=c.newcinsfx left join cycle_data as e on a.newcinsfx=e.newcinsfx 
left join tr_an_${riskprofile_new} as f on substr(a.newcinsfx,1,length(a.newcinsfx)-2)=f.cin_fna_done_on 
left join wlth_pen as g on a.cust_id = g.party_id where substr(c.RM_LAN_ID,1,4) != 'PVBK' and 
a.newcinsfx in (select CONCAT(CIN,SFX) from tciscust)
""")

TR_PORTCUBE_QV1.show()

+-------+-------------+--------+--------------+--------------+------------------+------------------+-----------+--------------+------------------+-------+-----------+---------------------+------------------+--------------+--------+-------+----------------+--------------+----------------------+------------+-------------+------------------+------------------+---------+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+---------------+---------------------------+-------------------------+---+-----------+--------+-------------------+---------------------+--------------------+----------------+------------------------+----------------------+---------------+----------+-----------------+--------+-------+---------+---------+-------------------+--------------+-----------------+-----------------+-----------------------+------------+-----------------------+--------------+-----------+------+------+------------+-------+-------+-------

##p238: Laxmi

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *
spark = SparkSession.builder.appName('sastopyspark').getOrCreate()

columns = ["party_id","newcinsfx","f_cust_inferred_wealth","f_infwealth_car","f_infwealth_divcashdistri",
           "f_infwealth_incometax","f_infwealth_property","f_infwealth_propertytax","f_infwealth_sharepurchase",
           "f_infwealth_sharesale","f_infwealth_yfpassets","amt_infwealth_car","amt_infwealth_divcashdistri",
           "amt_infwealth_incometax","amt_infwealth_property","amt_infwealth_propertytax","amt_infwealth_sharepurchase",
           "amt_infwealth_sharesale","amt_infwealth_yfpassets"]

data = [(1,2,3,4,5,7,8,9,0,6,7,0,4,3,8,6,9,3,5), (4,5,7,8,5,0,4,6,3,2,6,8,0,4,3,9,7,5,9)]

spark.conf.set("c360_consolmart",10)

cmktingbc360_consolmart = spark.createDataFrame(data=data,schema=columns)
cmktingbc360_consolmart.show()
cmktingbc360_consolmart.createOrReplaceTempView('cmktingb_10')


data = [[1],[2],[3],[4]]
columns = ["CUST_ID"]
snapshot_rev = spark.createDataFrame(data=data, schema=columns)
snapshot_rev.show()
snapshot_rev.createOrReplaceTempView('snapshot_rev')

+--------+---------+----------------------+---------------+-------------------------+---------------------+--------------------+-----------------------+-------------------------+---------------------+---------------------+-----------------+---------------------------+-----------------------+----------------------+-------------------------+---------------------------+-----------------------+-----------------------+
|party_id|newcinsfx|f_cust_inferred_wealth|f_infwealth_car|f_infwealth_divcashdistri|f_infwealth_incometax|f_infwealth_property|f_infwealth_propertytax|f_infwealth_sharepurchase|f_infwealth_sharesale|f_infwealth_yfpassets|amt_infwealth_car|amt_infwealth_divcashdistri|amt_infwealth_incometax|amt_infwealth_property|amt_infwealth_propertytax|amt_infwealth_sharepurchase|amt_infwealth_sharesale|amt_infwealth_yfpassets|
+--------+---------+----------------------+---------------+-------------------------+---------------------+--------------------+-----------------------+------------

In [ ]:
c360 = spark.sql("""

select party_id , newcinsfx , f_cust_inferred_wealth , f_infwealth_car , f_infwealth_divcashdistri , f_infwealth_incometax , 
f_infwealth_property , f_infwealth_propertytax , f_infwealth_sharepurchase , f_infwealth_sharesale , f_infwealth_yfpassets , 
0 as k_voc_complaints , 0 as k_voc_compliments , (amt_infwealth_car + amt_infwealth_divcashdistri + amt_infwealth_incometax +
amt_infwealth_property + amt_infwealth_propertytax + amt_infwealth_sharepurchase + amt_infwealth_sharesale + amt_infwealth_yfpassets) as 
total_inferred_aum from cmktingb_${c360_consolmart} where party_id in (select CUST_ID from snapshot_rev)

""")
c360.show()

+--------+---------+----------------------+---------------+-------------------------+---------------------+--------------------+-----------------------+-------------------------+---------------------+---------------------+----------------+-----------------+------------------+
|party_id|newcinsfx|f_cust_inferred_wealth|f_infwealth_car|f_infwealth_divcashdistri|f_infwealth_incometax|f_infwealth_property|f_infwealth_propertytax|f_infwealth_sharepurchase|f_infwealth_sharesale|f_infwealth_yfpassets|k_voc_complaints|k_voc_compliments|total_inferred_aum|
+--------+---------+----------------------+---------------+-------------------------+---------------------+--------------------+-----------------------+-------------------------+---------------------+---------------------+----------------+-----------------+------------------+
|       1|        2|                     3|              4|                        5|                    7|                   8|                      9|                 

### P45


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *
spark = SparkSession.builder.appName('sastopyspark').getOrCreate()

spark.conf.set('refresh_date',"13-12-2022")

data = [[ "OPT_IN","NR",10,0,"OPT_IN","NR",8,"OPT_IN","NR",0,"OPT_IN","WDL","13-12-2022",10,5,20],
        [None,"BR",10,0,"None","NR",8,"FGFD","TR",0,"OPT_IN","WDL","13-12-2022",10,5,20]]

columns = ["PDPA_tm_CD","dnc_tm_cd","dnc_validity_dt","f_tm_unsub","PDPA_sms_CD","dnc_sms_cd",
          "f_sms_unsub","PDPA_fax_CD","dnc_fax_cd","f_fax_unsub","PDPA_email_CD","PDPA_dm_CD",
           "as_of_dt","f_dm_unsub"," f_email_unsub","newcinsfx"]

BIP_VCBG_V_ADM_CAMPAIGN_SELECTION_D= spark.createDataFrame(data,columns)

BIP_VCBG_V_ADM_CAMPAIGN_SELECTION_D.show()

BIP_VCBG_V_ADM_CAMPAIGN_SELECTION_D.createOrReplaceTempView('BIP_VCBG_V_ADM_CAMPAIGN_SELECTION_D')


## BIP_VCBG_V_ADM

data = [[ "OPT_IN","NR",10,0,"OPT_IN","NR",8,"OPT_IN","NR",0,"OPT_IN","WDL","2020-12-03",10,5,20],
        ["None","BR",15,0,"None","NR",8,"FGFD","TR",0,"OPT_IN","WDL","2020-12-03",10,5,20]]

columns = ["PDPA_tm_CD","dnc_tm_cd","dnc_validity_dt","f_tm_unsub","PDPA_sms_CD","dnc_sms_cd",
           "f_sms_unsub","PDPA_fax_CD","dnc_fax_cd","f_fax_unsub","PDPA_email_CD","PDPA_dm_CD",
           "as_of_dt","f_dm_unsub","f_email_unsub","newcinsfx"]


BIP_VCBG_V_ADM= spark.createDataFrame(data,columns)
BIP_VCBG_V_ADM.show()
BIP_VCBG_V_ADM.createOrReplaceTempView('BIP_VCBG_V_ADM')

+----------+---------+---------------+----------+-----------+----------+-----------+-----------+----------+-----------+-------------+----------+----------+----------+--------------+---------+
|PDPA_tm_CD|dnc_tm_cd|dnc_validity_dt|f_tm_unsub|PDPA_sms_CD|dnc_sms_cd|f_sms_unsub|PDPA_fax_CD|dnc_fax_cd|f_fax_unsub|PDPA_email_CD|PDPA_dm_CD|  as_of_dt|f_dm_unsub| f_email_unsub|newcinsfx|
+----------+---------+---------------+----------+-----------+----------+-----------+-----------+----------+-----------+-------------+----------+----------+----------+--------------+---------+
|    OPT_IN|       NR|             10|         0|     OPT_IN|        NR|          8|     OPT_IN|        NR|          0|       OPT_IN|       WDL|13-12-2022|        10|             5|       20|
|      null|       BR|             10|         0|       None|        NR|          8|       FGFD|        TR|          0|       OPT_IN|       WDL|13-12-2022|        10|             5|       20|
+----------+---------+---------------+--

In [ ]:
tempVar = spark.sql("""
select max(dnc_validity_dt) from BIP_VCBG_V_ADM
""")

In [ ]:
tempVar.createOrReplaceTempView('tempVar')

In [ ]:
spark.sql("select * from tempVar").show()

+--------------------+
|max(dnc_validity_dt)|
+--------------------+
|                  15|
+--------------------+



In [ ]:
spark.conf.set('dnc_validity_dt',15)

In [ ]:
spark.sql("""

select newcinsfx, 
case when (PDPA_tm_CD = 'OPT_IN' or (PDPA_tm_CD is NULL and dnc_tm_cd = 'NR' and 
dnc_validity_dt = "${dnc_validity_dt}")) and f_tm_unsub = 0 then 1 when (PDPA_tm_CD = 'WDL' or (PDPA_tm_CD is NULL and dnc_tm_cd = 'DNC'
and dnc_validity_dt = "${dnc_validity_dt}")) and f_tm_unsub = 1 then 0 else null end as f_pdpa_tm from BIP_VCBG_V_ADM

""").show()

+---------+---------+
|newcinsfx|f_pdpa_tm|
+---------+---------+
|       20|        1|
|       20|     null|
+---------+---------+



In [ ]:
spark.sql("""
select newcinsfx, 
case when (PDPA_sms_CD = 'OPT_IN'
or (PDPA_sms_CD is NULL and dnc_sms_cd = 'NR' and dnc_validity_dt = "${dnc_validity_dt}")) and f_sms_unsub = 0 then 1 when (PDPA_sms_CD = 'WDL'
or (PDPA_sms_CD is NULL and dnc_sms_cd = 'DNC' and dnc_validity_dt = "${dnc_validity_dt}")) and f_sms_unsub = 1 then 0 e
lse null end as f_pdpa_sms from BIP_VCBG_V_ADM
""").show()

+---------+----------+
|newcinsfx|f_pdpa_sms|
+---------+----------+
|       20|      null|
|       20|      null|
+---------+----------+



In [ ]:
spark.sql("""

select newcinsfx,
case when (PDPA_fax_CD = 'OPT_IN' or (PDPA_fax_CD is NULL and dnc_fax_cd = 'NR'
and dnc_validity_dt = "${dnc_validity_dt}"))
and f_fax_unsub = 0 then 1 when (PDPA_fax_CD = 'WDL' or (PDPA_fax_CD is NULL
and dnc_fax_cd = 'DNC' and dnc_validity_dt = "${dnc_validity_dt}")) and f_fax_unsub = 1 
then 0 else null end as f_pdpa_fax from BIP_VCBG_V_ADM

""").show()




+---------+----------+
|newcinsfx|f_pdpa_fax|
+---------+----------+
|       20|         1|
|       20|      null|
+---------+----------+



In [ ]:
spark.sql("""
select newcinsfx, 
case when (PDPA_email_CD = 'OPT_IN' or (PDPA_email_CD is NULL and f_email_unsub = 0)) then 1
when (PDPA_email_CD = 'WDL' or (PDPA_email_CD is NULL and f_email_unsub = 1)) then 0 else null end as f_pdpa_email from BIP_VCBG_V_ADM
""").show()

+---------+------------+
|newcinsfx|f_pdpa_email|
+---------+------------+
|       20|           1|
|       20|           1|
+---------+------------+



In [ ]:
spark.sql("""

select newcinsfx, 
case when (PDPA_dm_CD = 'OPT_IN' or (PDPA_dm_CD is NULL and f_dm_unsub = 0)) then 1 when (PDPA_dm_CD = 'WDL' or 
(PDPA_dm_CD is NULL and f_dm_unsub = 1)) then 0 else null end as f_pdpa_dm from 
BIP_VCBG_V_ADM where as_of_dt = to_date("${refresh_date}",'dd-mm-yyyy') 

""").show()

+---------+---------+
|newcinsfx|f_pdpa_dm|
+---------+---------+
+---------+---------+



In [ ]:
# proc sql;
# create table CUST_IBG_RELATIONSHIP as
# select newcinsfx,
# case when f_ibg12 gt 0 then "Y"
# else "N" end as IBG_12,
# case when f_ibg34 gt 0 then "Y"
# else "N" end as IBG_34
# from CUST_IBG_RELATIONSHIP1;
# quit;



In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *
spark = SparkSession.builder.appName('sastopyspark').getOrCreate()

data = [[12, 2, 5],
        [14, 1, 6],
        [17, 3, 8],
        [15, 0, 1]]

columns = ["newcinsfx", "f_ibg12", "f_ibg34"]

CUST_IBG_RELATIONSHIP1= spark.createDataFrame(data,columns)
CUST_IBG_RELATIONSHIP1.show()
CUST_IBG_RELATIONSHIP1.createOrReplaceTempView('CUST_IBG_RELATIONSHIP1')

+---------+-------+-------+
|newcinsfx|f_ibg12|f_ibg34|
+---------+-------+-------+
|       12|      2|      5|
|       14|      1|      6|
|       17|      3|      8|
|       15|      0|      1|
+---------+-------+-------+



### New

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *
spark = SparkSession.builder.appName('sastopyspark').getOrCreate()

data = [[1,2,3,4,5,6,7], [2,3,5,7,8,9,4], [2,1,4,6,8,9,1], [12,4,6,7,8,9,4]]

columns = ["cdnumpol", "ctvrspol", "tcramdgs", "tcsubram", "default", "random", "option"]

spark.conf.set('kstec410',20)

stagedb_kstec410 = spark.createDataFrame(data,columns)

stagedb_kstec410.show()

stagedb_kstec410.createOrReplaceTempView('stagedb_20')

+--------+--------+--------+--------+-------+------+------+
|cdnumpol|ctvrspol|tcramdgs|tcsubram|default|random|option|
+--------+--------+--------+--------+-------+------+------+
|       1|       2|       3|       4|      5|     6|     7|
|       2|       3|       5|       7|      8|     9|     4|
|       2|       1|       4|       6|      8|     9|     1|
|      12|       4|       6|       7|      8|     9|     4|
+--------+--------+--------+--------+-------+------+------+



In [ ]:
def proc_sql(data):
  small_data = data.select(["cdnumpol", "ctvrspol", "tcramdgs", "tcsubram"])
  return small_data

myindex = proc_sql(data=stagedb_kstec410)
myindex.show()

+--------+--------+--------+--------+
|cdnumpol|ctvrspol|tcramdgs|tcsubram|
+--------+--------+--------+--------+
|       1|       2|       3|       4|
|       2|       3|       5|       7|
|       2|       1|       4|       6|
|      12|       4|       6|       7|
+--------+--------+--------+--------+



In [ ]:
spark.sql("""create index myindex on stagedb_${kstec410}(cdnumpol, ctvrspol, tcramdgs, tcsubram)""")

ParseException: ignored

p1

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *
spark = SparkSession.builder.appName('sastopyspark').getOrCreate()

data=[["AB","BB","Londonn1",4,'abcdefgdg',"DD","0000000000","0000000000"],
       ["AB","BB","Londonn1",4,'abcdefgdg',"DD","0000000000","0000000000"],
       ["AB","BB","Londonn1",4,'abcdefgdg',"DD","0000000000","0000000000"],
       ["AB","BB","Londonn1",4,'abcdefgdg',"DD","0000000000","0000000000"],
       ["AB","BB","Londonn1",4,'abcdefgdg',"DD","0000000000","0000000000"],
       ["AB","BB","Londonn1",4,'abcdefgdg',"DD","0000000000","0000000000"]]

columns=["cdprodte","cdprodco","cdelem","nurerelc","nocampfi","cdobjtp","cdmct","cdmcr"]

sourcedb_ktpa82t = spark.createDataFrame(data,columns)

sourcedb_ktpa82t.show()

sourcedb_ktpa82t.createOrReplaceTempView('sourcedb_ktpa82t')

+--------+--------+--------+--------+---------+-------+----------+----------+
|cdprodte|cdprodco|  cdelem|nurerelc| nocampfi|cdobjtp|     cdmct|     cdmcr|
+--------+--------+--------+--------+---------+-------+----------+----------+
|      AB|      BB|Londonn1|       4|abcdefgdg|     DD|0000000000|0000000000|
|      AB|      BB|Londonn1|       4|abcdefgdg|     DD|0000000000|0000000000|
|      AB|      BB|Londonn1|       4|abcdefgdg|     DD|0000000000|0000000000|
|      AB|      BB|Londonn1|       4|abcdefgdg|     DD|0000000000|0000000000|
|      AB|      BB|Londonn1|       4|abcdefgdg|     DD|0000000000|0000000000|
|      AB|      BB|Londonn1|       4|abcdefgdg|     DD|0000000000|0000000000|
+--------+--------+--------+--------+---------+-------+----------+----------+



In [4]:
ktpa82t_obj_au = spark.sql("""

select substr(ktpa82t.cdprodte,1,10) as cdprodte,
substr(ktpa82t.cdprodco,1,10) as cdprodco,
substr(ktpa82t.cdobjtp, 1,10) as cdobjtp,
substr(ktpa82t.cdelem,1,8) as cdelem,
ktpa82t.nurerelc as nurerelc,
substr(ktpa82t.nocampfi,1,8) as nocampfi,
substr(ktpa82t.cdmct,1,10) as cdmct ,
substr(ktpa82t.cdmcr,1,10) as cdmcr 
from sourcedb_ktpa82t as ktpa82t where 
(ktpa82t.cdprodte LIKE 'A%'and (ktpa82t.nurerelc != 0
 and (ktpa82t.cdobjtp != '0000000000'and (ktpa82t.cdmct == '0000000000'and (ktpa82t.cdprodco != '0000000000'and ktpa82t.cdmcr == '0000000000')))))
 order by ktpa82t.cdprodte, ktpa82t.cdprodco, ktpa82t.cdobjtp, ktpa82t.cdelem""")

ktpa82t_obj_au.show()

+--------+--------+-------+--------+--------+--------+----------+----------+
|cdprodte|cdprodco|cdobjtp|  cdelem|nurerelc|nocampfi|     cdmct|     cdmcr|
+--------+--------+-------+--------+--------+--------+----------+----------+
|      AB|      BB|     DD|Londonn1|       4|abcdefgd|0000000000|0000000000|
|      AB|      BB|     DD|Londonn1|       4|abcdefgd|0000000000|0000000000|
|      AB|      BB|     DD|Londonn1|       4|abcdefgd|0000000000|0000000000|
|      AB|      BB|     DD|Londonn1|       4|abcdefgd|0000000000|0000000000|
|      AB|      BB|     DD|Londonn1|       4|abcdefgd|0000000000|0000000000|
|      AB|      BB|     DD|Londonn1|       4|abcdefgd|0000000000|0000000000|
+--------+--------+-------+--------+--------+--------+----------+----------+



Manasa's Code

In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *
spark = SparkSession.builder.appName('sastopyspark').getOrCreate()

columns=['CDSINIES', 'CTNUPOIN', 'CDMCT', 'CDMCR', 'NUMOVIMI', 'TCTIPMOV', 'TCTIPMOA', 'IMMOVIMI', 'IMMOVDOL','TCCDPAG', 
         'FEOPERA', 'TCTIPRES', 'TCLASRES', 'TCDESSUB', 'CDMEDPAG', 'ESTADOMO', 'IMSALREB', 'IMSRBDOL', 'NUMOVITO', 
         'NUMIDTRA', 'TCCOPERA', 'NUMOVECO', 'CDTIPAGO','IMPCEC62','CDREFDET','CDREFCAB','NUAPUNTE','FECONTAB','IMDEDUCI','IMCOASEG']

columns = list(set(columns))

data=[list(range(len(columns)))]

STAGEDB_KSTSI165=spark.createDataFrame(data=data,schema=columns)

STAGEDB_KSTSI165.createOrReplaceTempView('STAGEDB_KSTSI160')

STAGEDB_KSTSI165.createOrReplaceTempView('stagedb_KTOM14T')

In [9]:
spark.sql("""SELECT KSTSI160.CDSINIES, LEFT(KSTSI160.CTNUPOIN,4),LEFT(KSTSI160.CDMCT,10),
LEFT(KSTSI160.CDMCR,10),LEFT(KSTSI160.NUMOVIMI,5),LEFT(KSTSI160.TCTIPMOV,1),LEFT(KSTSI160.TCTIPMOA,1),
LEFT(KSTSI160.IMMOVIMI,17.3),LEFT(KSTSI160.IMMOVDOL,17.3),LEFT(KSTSI160.TCCDPAG,3),LEFT(KSTSI160.FEOPERA,9),
LEFT(KSTSI160.TCTIPRES,1),LEFT(KSTSI160.TCLASRES,1),LEFT(KSTSI160.TCDESSUB,5),LEFT(KSTSI160.CDMEDPAG,2),
LEFT(KSTSI160.ESTADOMO,2),LEFT(KSTSI160.IMSALREB,17.3),LEFT(KSTSI160.IMSRBDOL,17.3),LEFT(KSTSI160.NUMOVITO,4),
LEFT(KSTSI160.NUMIDTRA,12),LEFT(KSTSI160.TCCOPERA,3),LEFT(KSTSI160.NUMOVECO,12),LEFT(KSTSI160.CDTIPAGO,2),
LEFT(KSTSI160.IMPCEC62,19.3),LEFT(KSTSI160.CDREFDET,16),LEFT(KSTSI160.CDREFCAB,10),LEFT(KSTSI160.NUAPUNTE,18),
LEFT(KSTSI160.FECONTAB,9), LEFT(KTOM14T.IMDEDUCI,17.3), LEFT(KTOM14T.IMCOASEG,17.3) 
from STAGEDB_KSTSI160 AS KSTSI160 left join stagedb_KTOM14T AS KTOM14T ON KSTSI160.CDSINIES = KTOM14T.CDSINIES
""").show()

+--------+-----------------+---------------+---------------+-----------------+-----------------+-----------------+--------------------+--------------------+----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+--------------------+--------------------+-----------------+------------------+-----------------+------------------+-----------------+--------------------+------------------+------------------+------------------+-----------------+--------------------+--------------------+
|CDSINIES|left(CTNUPOIN, 4)|left(CDMCT, 10)|left(CDMCR, 10)|left(NUMOVIMI, 5)|left(TCTIPMOV, 1)|left(TCTIPMOA, 1)|left(IMMOVIMI, 17.3)|left(IMMOVDOL, 17.3)|left(TCCDPAG, 3)|left(FEOPERA, 9)|left(TCTIPRES, 1)|left(TCLASRES, 1)|left(TCDESSUB, 5)|left(CDMEDPAG, 2)|left(ESTADOMO, 2)|left(IMSALREB, 17.3)|left(IMSRBDOL, 17.3)|left(NUMOVITO, 4)|left(NUMIDTRA, 12)|left(TCCOPERA, 3)|left(NUMOVECO, 12)|left(CDTIPAGO, 2)|left(IMPCEC62, 19.3)|left(CDREFDE